In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
all_train_snp_all = pd.read_csv("./csv/all_train_snp_selected.csv")
validation_snp_all = pd.read_csv("./csv/validation_snp_selected.csv")

In [4]:
X_train_all_ = all_train_snp_all.drop(columns=["Unnamed: 0", "target"]).to_numpy()
X_val_all_ = validation_snp_all.drop(columns=["Unnamed: 0", "target"]).to_numpy()
y_all_train = all_train_snp_all["target"] - 1
y_val = validation_snp_all["target"] - 1

In [8]:
all_train_pgs = pd.read_csv("./pgs_results_calculated/all_train_pgs.csv").drop(columns=["FID", "IID", "y"])
validation_pgs = pd.read_csv("./pgs_results_calculated/validation_pgs.csv").drop(columns=["FID", "IID", "y"])

In [39]:
X_train_pgs = all_train_pgs.to_numpy()
X_val_pgs = validation_pgs.to_numpy()
ss = StandardScaler()
X_train_pgs = ss.fit_transform(X_train_pgs)
X_val_pgs = ss.transform(X_val_pgs)

In [40]:
train_snps = []
test_snps = []
train_pgss = []
test_pgss = []
for i in range(1, 6):
    train_snp = pd.read_csv(f"./csv/train_{i}_snp_selected.csv")
    train_snps.append(train_snp)
    test_snp = pd.read_csv(f"./csv/test_{i}_snp_selected.csv")
    test_snps.append(test_snp)
    ss = StandardScaler()
    train_pgs = ss.fit_transform(pd.read_csv(f"./pgs_results_calculated/train_{i}_pgs.csv").drop(columns=["FID", "IID", "y"]).to_numpy())
    train_pgss.append(train_pgs)
    test_pgs = ss.transform(pd.read_csv(f"./pgs_results_calculated/test_{i}_pgs.csv").drop(columns=["FID", "IID", "y"]).to_numpy())
    test_pgss.append(test_pgs)

Регрессия на всех SNP

In [41]:
ss = StandardScaler()
X_train_all = ss.fit_transform(np.concatenate([X_train_all_, X_train_pgs], axis=1))
X_val_all = ss.transform(np.concatenate([X_val_all_, X_val_pgs], axis=1))

model_gb = LogisticRegression()
model_gb.fit(X_train_all, y_all_train)

y_pred_proba = model_gb.predict_proba(X_val_all)[:, 1]

roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC bigset: {roc_auc:.4f}')

ROC-AUC bigset: 0.7321


In [42]:
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = np.concatenate([train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy(), train_pgs], axis=1)
    X_val_ = np.concatenate([test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy(), test_pgs], axis=1)
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)

    model_gb = LogisticRegression()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]

    roc_auc = roc_auc_score(y_test, y_pred_proba)
    rocs.append(roc_auc)
    print(f'ROC-AUC: {roc_auc:.4f}')
print("среднее", np.mean(rocs))

ROC-AUC: 0.6885
ROC-AUC: 0.7077
ROC-AUC: 0.7030
ROC-AUC: 0.6694
ROC-AUC: 0.7541
среднее 0.7045301483766652


Градиентный бустинг на всех SNP

In [43]:
ss = StandardScaler()
X_train_all = ss.fit_transform(np.concatenate([X_train_all_, X_train_pgs], axis=1))
X_val_all = ss.transform(np.concatenate([X_val_all_, X_val_pgs], axis=1))

model_gb = GradientBoostingClassifier()
model_gb.fit(X_train_all, y_all_train)

y_pred_proba = model_gb.predict_proba(X_val_all)[:, 1]

roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC bigset: {roc_auc:.4f}')

ROC-AUC bigset: 0.7188


In [44]:
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = np.concatenate([train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy(), train_pgs], axis=1)
    X_val_ = np.concatenate([test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy(), test_pgs], axis=1)
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)

    model_gb = GradientBoostingClassifier()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]

    roc_auc = roc_auc_score(y_test, y_pred_proba)
    rocs.append(roc_auc)
    print(f'ROC-AUC: {roc_auc:.4f}')
print("среднее", np.mean(rocs))

ROC-AUC: 0.7041
ROC-AUC: 0.7433
ROC-AUC: 0.7327
ROC-AUC: 0.6941
ROC-AUC: 0.7444
среднее 0.7237413260365753


SVC на всех SNP

In [45]:
ss = StandardScaler()
X_train_all = ss.fit_transform(np.concatenate([X_train_all_, X_train_pgs], axis=1))
X_val_all = ss.transform(np.concatenate([X_val_all_, X_val_pgs], axis=1))

model_gb = SVC(probability=True)
model_gb.fit(X_train_all, y_all_train)

y_pred_proba = model_gb.predict_proba(X_val_all)[:, 1]

roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC bigset: {roc_auc:.4f}')

ROC-AUC bigset: 0.7374


In [46]:
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = np.concatenate([train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy(), train_pgs], axis=1)
    X_val_ = np.concatenate([test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy(), test_pgs], axis=1)
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)

    model_gb = SVC(probability=True)
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]

    roc_auc = roc_auc_score(y_test, y_pred_proba)
    rocs.append(roc_auc)
    print(f'ROC-AUC: {roc_auc:.4f}')
print("среднее", np.mean(rocs))

ROC-AUC: 0.7171
ROC-AUC: 0.7331
ROC-AUC: 0.7352
ROC-AUC: 0.7106
ROC-AUC: 0.7634
среднее 0.7318608849429276


Полносвязная сеть на всех snp

In [47]:
ss = StandardScaler()
X_train_all = ss.fit_transform(np.concatenate([X_train_all_, X_train_pgs], axis=1))
X_val_all = ss.transform(np.concatenate([X_val_all_, X_val_pgs], axis=1))

class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

X_train_tensor = torch.FloatTensor(X_train_all)
y_train_tensor = torch.FloatTensor(y_all_train.values).reshape(-1, 1)

model = Net(X_train_all.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model.train()
for epoch in range(10):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

model.eval()
with torch.no_grad():
    X_val_tensor = torch.FloatTensor(X_val_all)
    y_pred_proba = model(X_val_tensor).numpy().flatten()

roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


ROC-AUC autoencoded bigset: 0.7666


In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = np.concatenate([train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy(), train_pgs], axis=1)
    X_val_ = np.concatenate([test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy(), test_pgs], axis=1)
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)

    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
    
    model = Net(X_train.shape[1])
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    model.train()
    for epoch in range(10):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
    
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.FloatTensor(X_val)
        y_pred_proba = model(X_val_tensor).numpy().flatten()
    
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    rocs.append(roc_auc)
    print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7266
ROC-AUC autoencoded: 0.7502
ROC-AUC autoencoded: 0.7525
ROC-AUC autoencoded: 0.6956
ROC-AUC autoencoded: 0.7383
среднее 0.7326409170129168


Блок обучения с использованием классического автоэнкодера

С использованием логистической регрессии для классификации

In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import optuna
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim=128, dropout_rate=0.2):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.LeakyReLU(0.2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, input_dim)
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent
    
def train_autoencoder(model, X_train, epochs, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, _ in train_loader:
            optimizer.zero_grad()
            reconstructed, _ = model(batch_x)
            loss = criterion(reconstructed, batch_x)
            loss.backward()
            optimizer.step()
    
    return model

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        _, latent = model(X_tensor)
    return latent.cpu().numpy()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    C = trial.suggest_float('C', 1e-3, 10.0, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    autoencoder = Autoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
    autoencoder = train_autoencoder(autoencoder, X_train, epochs, batch_size, lr)
    
    X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
    X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
    
    logreg = LogisticRegression(C=C, max_iter=1000)
    logreg.fit(X_train_latent, y_train)
    
    y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    
    return roc_auc

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = Autoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

logreg = LogisticRegression(C=best_params['C'], max_iter=1000)
logreg.fit(X_train_latent, y_all_train)

y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 20:16:05,668] A new study created in memory with name: no-name-ceeb740f-a317-4942-a61f-329dd8ea1193
[I 2025-05-10 20:16:12,222] Trial 0 finished with value: 0.7621923165401425 and parameters: {'latent_dim': 37, 'hidden_dim': 178, 'lr': 0.009158291855730935, 'epochs': 46, 'C': 0.3182338683217116, 'dropout_rate': 0.483964389594041}. Best is trial 0 with value: 0.7621923165401425.
[I 2025-05-10 20:16:14,651] Trial 1 finished with value: 0.7631700023004371 and parameters: {'latent_dim': 34, 'hidden_dim': 143, 'lr': 0.0019095555895441753, 'epochs': 19, 'C': 0.3113717523856937, 'dropout_rate': 0.14855696670680368}. Best is trial 1 with value: 0.7631700023004371.
[I 2025-05-10 20:16:20,958] Trial 2 finished with value: 0.7661605705083967 and parameters: {'latent_dim': 34, 'hidden_dim': 164, 'lr': 0.0008534256270681129, 'epochs': 48, 'C': 0.0017160381632241107, 'dropout_rate': 0.32109264853624353}. Best is trial 2 with value: 0.7661605705083967.
[I 2025-05-10 20:16:23,060] Trial 

ROC-AUC autoencoded bigset: 0.7731


In [52]:
best_params

{'latent_dim': 73,
 'hidden_dim': 66,
 'lr': 0.00036296363705733587,
 'epochs': 22,
 'C': 0.010098480760178918,
 'dropout_rate': 0.2289821605173353}

In [55]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)

    autoencoder = Autoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
    autoencoder = train_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'])
    
    X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
    X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
    
    logreg = LogisticRegression(C=best_params['C'], max_iter=1000)
    logreg.fit(X_train_latent, y_train)
    
    y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    rocs.append(roc_auc)
    print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7586
ROC-AUC autoencoded: 0.7777
ROC-AUC autoencoded: 0.7416
ROC-AUC autoencoded: 0.7053
ROC-AUC autoencoded: 0.7600
среднее 0.7486588660295539


с градиентным бустингом

In [57]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    # Параметры автоэнкодера
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    # Параметры градиентного бустинга
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    rocs = []
    for i in range(3):
        autoencoder = Autoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_autoencoder(autoencoder, X_train, epochs, batch_size, lr)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        logreg = GradientBoostingClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = Autoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

logreg = GradientBoostingClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
logreg.fit(X_train_latent, y_all_train)

y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 20:26:39,860] A new study created in memory with name: no-name-b9ad9930-e30e-41a4-9e81-900592c76f9a
[I 2025-05-10 20:27:50,576] Trial 0 finished with value: 0.7423702170078981 and parameters: {'latent_dim': 90, 'hidden_dim': 170, 'lr': 0.00010981442731189312, 'epochs': 17, 'dropout_rate': 0.46939251449007113, 'n_estimators': 254, 'max_depth': 5, 'learning_rate': 0.017113280095762472, 'subsample': 0.7140088324766143, 'min_samples_split': 16, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7423702170078981.
[I 2025-05-10 20:30:50,170] Trial 1 finished with value: 0.737941875623035 and parameters: {'latent_dim': 92, 'hidden_dim': 187, 'lr': 0.008093118002276292, 'epochs': 48, 'dropout_rate': 0.3681739947203867, 'n_estimators': 433, 'max_depth': 8, 'learning_rate': 0.011373414878795882, 'subsample': 0.8748184994158401, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7423702170078981.
[I 2025-05-10 20:31:27,679] Trial 2 finished with value: 

ROC-AUC autoencoded bigset: 0.7493


In [58]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = Autoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        logreg = GradientBoostingClassifier(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            subsample=best_params['subsample'],
            min_samples_split=best_params['min_samples_split'],
            min_samples_leaf=best_params['min_samples_leaf'],
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))

ROC-AUC autoencoded: 0.7387
ROC-AUC autoencoded: 0.7492
ROC-AUC autoencoded: 0.7468
ROC-AUC autoencoded: 0.7401
ROC-AUC autoencoded: 0.7440
ROC-AUC autoencoded: 0.7441
ROC-AUC autoencoded: 0.7375
ROC-AUC autoencoded: 0.7411
ROC-AUC autoencoded: 0.7239
ROC-AUC autoencoded: 0.7376
ROC-AUC autoencoded: 0.7373
ROC-AUC autoencoded: 0.7369
ROC-AUC autoencoded: 0.7414
ROC-AUC autoencoded: 0.7349
ROC-AUC autoencoded: 0.7366
ROC-AUC autoencoded: 0.7129
ROC-AUC autoencoded: 0.7136
ROC-AUC autoencoded: 0.7172
ROC-AUC autoencoded: 0.7167
ROC-AUC autoencoded: 0.7226
ROC-AUC autoencoded: 0.7455
ROC-AUC autoencoded: 0.7495
ROC-AUC autoencoded: 0.7479
ROC-AUC autoencoded: 0.7468
ROC-AUC autoencoded: 0.7535
среднее 0.7366493156384593


Теперь с SVC

In [59]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    # Параметры автоэнкодера
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    # Параметры SVC
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    
    rocs = []
    for i in range(3):
        autoencoder = Autoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_autoencoder(autoencoder, X_train, epochs, batch_size, lr)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        svc = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            degree=degree,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = Autoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

svc = SVC(
    C=best_params['C'],
    kernel=best_params['kernel'],
    gamma=best_params['gamma'] if best_params['kernel'] in ['rbf', 'poly', 'sigmoid'] else 'scale',
    degree=best_params['degree'] if best_params['kernel'] == 'poly' else 3,
    probability=True,
    random_state=42
)
svc.fit(X_train_latent, y_all_train)

y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 20:45:25,253] A new study created in memory with name: no-name-31ec2f1c-0f36-4728-bccc-a51b3f27b8c2
[I 2025-05-10 20:45:32,051] Trial 0 finished with value: 0.6033279656468062 and parameters: {'latent_dim': 24, 'hidden_dim': 66, 'lr': 0.0004236092982241249, 'epochs': 24, 'dropout_rate': 0.3034119355588364, 'C': 5.20596690472872, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.6033279656468062.
[I 2025-05-10 20:45:48,373] Trial 1 finished with value: 0.741660915573959 and parameters: {'latent_dim': 65, 'hidden_dim': 248, 'lr': 0.0015545428444563034, 'epochs': 50, 'dropout_rate': 0.2682758189179693, 'C': 0.2469476376907018, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.741660915573959.
[I 2025-05-10 20:45:59,654] Trial 2 finished with value: 0.6659957058507783 and parameters: {'latent_dim': 42, 'hidden_dim': 166, 'lr': 0.00013076216775538712, 'epochs': 36, 'dropout_rate': 0.3447443841718676, 'C': 0.45067734351800826, 'kernel': 'sigmoi

ROC-AUC autoencoded bigset: 0.7694


In [60]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = Autoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        svc = SVC(
            C=best_params['C'],
            kernel=best_params['kernel'],
            gamma=best_params['gamma'] if best_params['kernel'] in ['rbf', 'poly', 'sigmoid'] else 'scale',
            degree=best_params['degree'] if best_params['kernel'] == 'poly' else 3,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7000
ROC-AUC autoencoded: 0.7129
ROC-AUC autoencoded: 0.7199
ROC-AUC autoencoded: 0.7076
ROC-AUC autoencoded: 0.7000
ROC-AUC autoencoded: 0.7299
ROC-AUC autoencoded: 0.7367
ROC-AUC autoencoded: 0.7548
ROC-AUC autoencoded: 0.7340
ROC-AUC autoencoded: 0.7497
ROC-AUC autoencoded: 0.7378
ROC-AUC autoencoded: 0.7417
ROC-AUC autoencoded: 0.7468
ROC-AUC autoencoded: 0.7363
ROC-AUC autoencoded: 0.7487
ROC-AUC autoencoded: 0.6503
ROC-AUC autoencoded: 0.6516
ROC-AUC autoencoded: 0.6529
ROC-AUC autoencoded: 0.6742
ROC-AUC autoencoded: 0.6515
ROC-AUC autoencoded: 0.7350
ROC-AUC autoencoded: 0.7408
ROC-AUC autoencoded: 0.7361
ROC-AUC autoencoded: 0.7446
ROC-AUC autoencoded: 0.7159
среднее 0.7164051675909022


С MLP

In [64]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate=0.2):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

def train_mlp(model, X_train, y_train, epochs, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    return model

def predict_mlp(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        predictions = model(X_tensor)
    
    return predictions.cpu().numpy().flatten()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    # Параметры автоэнкодера
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim_ae = trial.suggest_int('hidden_dim_ae', 64, 256)
    lr_ae = trial.suggest_float('lr_ae', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs_ae = trial.suggest_int('epochs_ae', 10, 50)
    dropout_rate_ae = trial.suggest_float('dropout_rate_ae', 0.1, 0.5)
    
    # Параметры MLP
    hidden_dim_mlp = trial.suggest_int('hidden_dim_mlp', 32, 128)
    lr_mlp = trial.suggest_float('lr_mlp', 1e-4, 1e-2, log=True)
    batch_size_mlp = trial.suggest_categorical('batch_size_mlp', [16, 32, 64])
    epochs_mlp = trial.suggest_int('epochs_mlp', 10, 50)
    dropout_rate_mlp = trial.suggest_float('dropout_rate_mlp', 0.1, 0.5)
    
    rocs = []
    for i in range(3):
        autoencoder = Autoencoder(X_train.shape[1], latent_dim, hidden_dim_ae, dropout_rate_ae)
        autoencoder = train_autoencoder(autoencoder, X_train, epochs_ae, batch_size, lr_ae)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        mlp = MLP(X_train_latent.shape[1], hidden_dim_mlp, dropout_rate_mlp)
        mlp = train_mlp(mlp, X_train_latent, y_train, epochs_mlp, batch_size_mlp, lr_mlp)
        
        y_pred_proba = predict_mlp(mlp, X_val_latent)
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = Autoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
autoencoder = train_autoencoder(autoencoder, X_train_all, best_params['epochs_ae'], 32, best_params['lr_ae'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

mlp = MLP(X_train_latent.shape[1], best_params['hidden_dim_mlp'], best_params['dropout_rate_mlp'])
mlp = train_mlp(mlp, X_train_latent, y_all_train, best_params['epochs_mlp'], best_params['batch_size_mlp'], best_params['lr_mlp'])

y_pred_proba = predict_mlp(mlp, X_val_latent)
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 21:07:17,646] A new study created in memory with name: no-name-0401ac9f-d6fc-4791-a14a-08be54858576
[I 2025-05-10 21:07:33,032] Trial 0 finished with value: 0.7623840196303964 and parameters: {'latent_dim': 13, 'hidden_dim_ae': 83, 'lr_ae': 0.0016714360488584885, 'epochs_ae': 35, 'dropout_rate_ae': 0.4100147384409282, 'hidden_dim_mlp': 35, 'lr_mlp': 0.0010455354639130117, 'batch_size_mlp': 32, 'epochs_mlp': 15, 'dropout_rate_mlp': 0.38976174101096406}. Best is trial 0 with value: 0.7623840196303964.
[I 2025-05-10 21:07:57,537] Trial 1 finished with value: 0.7208227896633693 and parameters: {'latent_dim': 90, 'hidden_dim_ae': 254, 'lr_ae': 0.0010433857832760631, 'epochs_ae': 47, 'dropout_rate_ae': 0.13995367502606093, 'hidden_dim_mlp': 72, 'lr_mlp': 0.0015551454924189955, 'batch_size_mlp': 64, 'epochs_mlp': 46, 'dropout_rate_mlp': 0.21759582506376032}. Best is trial 0 with value: 0.7623840196303964.
[I 2025-05-10 21:08:08,345] Trial 2 finished with value: 0.733877770109654

ROC-AUC autoencoded bigset: 0.7754


In [66]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = Autoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
        autoencoder = train_autoencoder(autoencoder, X_train, best_params['epochs_ae'], 32, best_params['lr_ae'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        mlp = MLP(X_train_latent.shape[1], best_params['hidden_dim_mlp'], best_params['dropout_rate_mlp'])
        mlp = train_mlp(mlp, X_train_latent, y_train, best_params['epochs_mlp'], best_params['batch_size_mlp'], best_params['lr_mlp'])
        
        y_pred_proba = predict_mlp(mlp, X_val_latent)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7458
ROC-AUC autoencoded: 0.7454
ROC-AUC autoencoded: 0.7434
ROC-AUC autoencoded: 0.7462
ROC-AUC autoencoded: 0.7522
ROC-AUC autoencoded: 0.7556
ROC-AUC autoencoded: 0.7658
ROC-AUC autoencoded: 0.7728
ROC-AUC autoencoded: 0.7601
ROC-AUC autoencoded: 0.7641
ROC-AUC autoencoded: 0.7344
ROC-AUC autoencoded: 0.7297
ROC-AUC autoencoded: 0.7332
ROC-AUC autoencoded: 0.7266
ROC-AUC autoencoded: 0.7490
ROC-AUC autoencoded: 0.7106
ROC-AUC autoencoded: 0.7196
ROC-AUC autoencoded: 0.7102
ROC-AUC autoencoded: 0.6862
ROC-AUC autoencoded: 0.7038
ROC-AUC autoencoded: 0.7548
ROC-AUC autoencoded: 0.7600
ROC-AUC autoencoded: 0.7597
ROC-AUC autoencoded: 0.7496
ROC-AUC autoencoded: 0.7533
среднее 0.7412807902350896


Блок с классическим автоэнкодером с второй классифицирующей головой

логситическая регрессия

In [67]:
class ClassifyingAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim=128, dropout_rate=0.2):
        super(ClassifyingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.LeakyReLU(0.2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, input_dim)
        )
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        classification = self.classifier(latent)
        return reconstructed, latent, classification

def train_classifying_autoencoder(model, X_train, y_train, epochs, batch_size, lr, recon_weight, class_weight):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    recon_criterion = nn.MSELoss()
    class_criterion = nn.BCELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            reconstructed, _, classification = model(batch_x)
            
            recon_loss = recon_criterion(reconstructed, batch_x)
            class_loss = class_criterion(classification, batch_y)
            
            loss = recon_weight * recon_loss + class_weight * class_loss
            loss.backward()
            optimizer.step()
    
    return model

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        _, latent, _ = model(X_tensor)
    return latent.cpu().numpy()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    # Параметры автоэнкодера
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    # Веса для объединенной функции потерь
    recon_weight = trial.suggest_float('recon_weight', 0.1, 0.9)
    class_weight = 1.0 - recon_weight
    
    # Параметры логистической регрессии
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_classifying_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, recon_weight, class_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        logreg = LogisticRegression(C=C, max_iter=1000, random_state=42)
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_classifying_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, 
    best_params['lr'], best_params['recon_weight'], 1.0 - best_params['recon_weight']
)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

logreg = LogisticRegression(C=best_params['C'], max_iter=1000, random_state=42)
logreg.fit(X_train_latent, y_all_train)

y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')

[I 2025-05-10 21:15:02,141] A new study created in memory with name: no-name-1fbe59d4-ba18-4392-ad8b-70f945632178
[I 2025-05-10 21:15:11,972] Trial 0 finished with value: 0.5823364772640135 and parameters: {'latent_dim': 85, 'hidden_dim': 144, 'lr': 0.0008863120405804203, 'epochs': 27, 'dropout_rate': 0.2927137755142506, 'recon_weight': 0.17832524088973134, 'C': 0.08417701865571019}. Best is trial 0 with value: 0.5823364772640135.
[I 2025-05-10 21:15:21,225] Trial 1 finished with value: 0.6388697185798635 and parameters: {'latent_dim': 35, 'hidden_dim': 245, 'lr': 0.003936583874037625, 'epochs': 23, 'dropout_rate': 0.22834626110558812, 'recon_weight': 0.33331948767221525, 'C': 0.014186192824981014}. Best is trial 1 with value: 0.6388697185798635.
[I 2025-05-10 21:15:38,408] Trial 2 finished with value: 0.5821064335557089 and parameters: {'latent_dim': 95, 'hidden_dim': 171, 'lr': 0.0009131576242124419, 'epochs': 46, 'dropout_rate': 0.32324701685855695, 'recon_weight': 0.676823588843178

ROC-AUC autoencoded bigset: 0.7737


In [68]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_classifying_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs'], 32, 
            best_params['lr'], best_params['recon_weight'], 1.0 - best_params['recon_weight']
        )
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        logreg = LogisticRegression(C=best_params['C'], max_iter=1000, random_state=42)
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7347
ROC-AUC autoencoded: 0.7184
ROC-AUC autoencoded: 0.7471
ROC-AUC autoencoded: 0.7488
ROC-AUC autoencoded: 0.7311
ROC-AUC autoencoded: 0.7357
ROC-AUC autoencoded: 0.7563
ROC-AUC autoencoded: 0.7718
ROC-AUC autoencoded: 0.7609
ROC-AUC autoencoded: 0.7791
ROC-AUC autoencoded: 0.7411
ROC-AUC autoencoded: 0.7303
ROC-AUC autoencoded: 0.7423
ROC-AUC autoencoded: 0.7305
ROC-AUC autoencoded: 0.7375
ROC-AUC autoencoded: 0.6755
ROC-AUC autoencoded: 0.6801
ROC-AUC autoencoded: 0.6968
ROC-AUC autoencoded: 0.6970
ROC-AUC autoencoded: 0.6938
ROC-AUC autoencoded: 0.7461
ROC-AUC autoencoded: 0.7597
ROC-AUC autoencoded: 0.7546
ROC-AUC autoencoded: 0.7575
ROC-AUC autoencoded: 0.7557
среднее 0.7352955008279881


Градиентный бустинг

In [69]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    # Параметры автоэнкодера
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    # Веса для объединенной функции потерь
    recon_weight = trial.suggest_float('recon_weight', 0.1, 0.9)
    class_weight = 1.0 - recon_weight
    
    # Параметры градиентного бустинга
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_classifying_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, recon_weight, class_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_classifying_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, 
    best_params['lr'], best_params['recon_weight'], 1.0 - best_params['recon_weight']
)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

gb = GradientBoostingClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
gb.fit(X_train_latent, y_all_train)

y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')

[I 2025-05-10 21:20:02,907] A new study created in memory with name: no-name-873325c3-fdc9-4ca8-944d-e7093de95b80
[I 2025-05-10 21:20:52,520] Trial 0 finished with value: 0.6894793344068706 and parameters: {'latent_dim': 69, 'hidden_dim': 66, 'lr': 0.0003592453319438576, 'epochs': 47, 'dropout_rate': 0.33676841219726805, 'recon_weight': 0.6930921587379049, 'n_estimators': 117, 'max_depth': 4, 'learning_rate': 0.07954057275621372, 'subsample': 0.9665096632357344, 'min_samples_split': 19, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.6894793344068706.
[I 2025-05-10 21:21:47,031] Trial 1 finished with value: 0.5653899240855762 and parameters: {'latent_dim': 41, 'hidden_dim': 120, 'lr': 0.006032840348185214, 'epochs': 46, 'dropout_rate': 0.25689892335937814, 'recon_weight': 0.7083611486364733, 'n_estimators': 266, 'max_depth': 5, 'learning_rate': 0.09376732490536899, 'subsample': 0.6137562140870056, 'min_samples_split': 6, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.68947

ROC-AUC autoencoded bigset: 0.7109


In [70]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_classifying_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs'], 32, 
            best_params['lr'], best_params['recon_weight'], 1.0 - best_params['recon_weight']
        )
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            subsample=best_params['subsample'],
            min_samples_split=best_params['min_samples_split'],
            min_samples_leaf=best_params['min_samples_leaf'],
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7130
ROC-AUC autoencoded: 0.7308
ROC-AUC autoencoded: 0.7045
ROC-AUC autoencoded: 0.7254
ROC-AUC autoencoded: 0.7050
ROC-AUC autoencoded: 0.6957
ROC-AUC autoencoded: 0.7331
ROC-AUC autoencoded: 0.7162
ROC-AUC autoencoded: 0.6838
ROC-AUC autoencoded: 0.6922
ROC-AUC autoencoded: 0.7270
ROC-AUC autoencoded: 0.7544
ROC-AUC autoencoded: 0.7289
ROC-AUC autoencoded: 0.7229
ROC-AUC autoencoded: 0.7488
ROC-AUC autoencoded: 0.6799
ROC-AUC autoencoded: 0.6886
ROC-AUC autoencoded: 0.6977
ROC-AUC autoencoded: 0.6932
ROC-AUC autoencoded: 0.7109
ROC-AUC autoencoded: 0.7271
ROC-AUC autoencoded: 0.7413
ROC-AUC autoencoded: 0.7204
ROC-AUC autoencoded: 0.7452
ROC-AUC autoencoded: 0.7267
среднее 0.7165108476697865


SVM

In [71]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    recon_weight = trial.suggest_float('recon_weight', 0.1, 0.9)
    class_weight = 1.0 - recon_weight
    
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_classifying_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, recon_weight, class_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        svc = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            degree=degree,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_classifying_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, 
    best_params['lr'], best_params['recon_weight'], 1.0 - best_params['recon_weight']
)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

svc = SVC(
    C=best_params['C'],
    kernel=best_params['kernel'],
    gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
    degree=best_params['degree'] if 'degree' in best_params else 3,
    probability=True,
    random_state=42
)
svc.fit(X_train_latent, y_all_train)

y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 21:44:58,121] A new study created in memory with name: no-name-33035a43-041b-445a-bfd0-f0ace4206c7f
[I 2025-05-10 21:45:03,912] Trial 0 finished with value: 0.7150046008741661 and parameters: {'latent_dim': 92, 'hidden_dim': 174, 'lr': 0.00040293924170599306, 'epochs': 12, 'dropout_rate': 0.21647311795149698, 'recon_weight': 0.7822068721387879, 'C': 0.11899817977582994, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.7150046008741661.
[I 2025-05-10 21:45:08,525] Trial 1 finished with value: 0.6729545280269917 and parameters: {'latent_dim': 89, 'hidden_dim': 79, 'lr': 0.0031839583742939775, 'epochs': 11, 'dropout_rate': 0.19706146089054083, 'recon_weight': 0.7220418494244706, 'C': 1.1556672891774213, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.7150046008741661.
[I 2025-05-10 21:45:17,007] Trial 2 finished with value: 0.608695652173913 and parameters: {'latent_dim': 65, 'hidden_dim': 221, 'lr': 0.00559228779234471, 'epochs': 20, 'dropo

ROC-AUC autoencoded bigset: 0.7262


In [72]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_classifying_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs'], 32, 
            best_params['lr'], best_params['recon_weight'], 1.0 - best_params['recon_weight']
        )
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        svc = SVC(
            C=best_params['C'],
            kernel=best_params['kernel'],
            gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
            degree=best_params['degree'] if 'degree' in best_params else 3,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7171
ROC-AUC autoencoded: 0.7040
ROC-AUC autoencoded: 0.7093
ROC-AUC autoencoded: 0.6979
ROC-AUC autoencoded: 0.7169
ROC-AUC autoencoded: 0.7319
ROC-AUC autoencoded: 0.7223
ROC-AUC autoencoded: 0.7173
ROC-AUC autoencoded: 0.7348
ROC-AUC autoencoded: 0.7126
ROC-AUC autoencoded: 0.7036
ROC-AUC autoencoded: 0.7094
ROC-AUC autoencoded: 0.7252
ROC-AUC autoencoded: 0.7168
ROC-AUC autoencoded: 0.7078
ROC-AUC autoencoded: 0.6963
ROC-AUC autoencoded: 0.7094
ROC-AUC autoencoded: 0.6949
ROC-AUC autoencoded: 0.7088
ROC-AUC autoencoded: 0.6979
ROC-AUC autoencoded: 0.7568
ROC-AUC autoencoded: 0.7491
ROC-AUC autoencoded: 0.7182
ROC-AUC autoencoded: 0.7531
ROC-AUC autoencoded: 0.7357
среднее 0.7178787651477744


MLP

In [73]:
class CombinedClassifier(nn.Module):
    def __init__(self, autoencoder, pgs_input_dim, hidden_dim=64, dropout_rate=0.2):
        super(CombinedClassifier, self).__init__()
        self.autoencoder = autoencoder
        self.pgs_branch = nn.Sequential(
            nn.Linear(pgs_input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2)
        )
        latent_dim = list(autoencoder.classifier[0].parameters())[0].shape[1]
        self.combined_classifier = nn.Sequential(
            nn.Linear(hidden_dim // 2 + latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x_snp, x_pgs):
        _, latent, _ = self.autoencoder(x_snp)
        pgs_features = self.pgs_branch(x_pgs)
        combined = torch.cat([latent, pgs_features], dim=1)
        output = self.combined_classifier(combined)
        return output

def train_combined_classifier(model, X_train_snp, X_train_pgs, y_train, epochs_freeze, epochs_unfreeze, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    X_train_snp_tensor = torch.FloatTensor(X_train_snp).to(device)
    X_train_pgs_tensor = torch.FloatTensor(X_train_pgs).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_snp_tensor, X_train_pgs_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    criterion = nn.BCELoss()
    
    # Этап 1: Заморозить автоэнкодер и его голову
    for param in model.autoencoder.parameters():
        param.requires_grad = False
    
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    model.train()
    for epoch in range(epochs_freeze):
        for batch_snp, batch_pgs, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_snp, batch_pgs)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    # Этап 2: Разморозить всё и обучать вместе
    for param in model.autoencoder.parameters():
        param.requires_grad = True
    
    optimizer = optim.Adam(model.parameters(), lr=lr/10)  # Уменьшаем скорость обучения для всей сети
    
    for epoch in range(epochs_unfreeze):
        for batch_snp, batch_pgs, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_snp, batch_pgs)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    return model

def predict_combined(model, X_snp, X_pgs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    X_snp_tensor = torch.FloatTensor(X_snp).to(device)
    X_pgs_tensor = torch.FloatTensor(X_pgs).to(device)
    
    with torch.no_grad():
        predictions = model(X_snp_tensor, X_pgs_tensor)
    
    return predictions.cpu().numpy().flatten()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim_ae = trial.suggest_int('hidden_dim_ae', 64, 256)
    hidden_dim_combined = trial.suggest_int('hidden_dim_combined', 32, 128)
    lr_ae = trial.suggest_float('lr_ae', 1e-4, 1e-2, log=True)
    lr_combined = trial.suggest_float('lr_combined', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs_ae = trial.suggest_int('epochs_ae', 10, 50)
    epochs_freeze = trial.suggest_int('epochs_freeze', 5, 20)
    epochs_unfreeze = trial.suggest_int('epochs_unfreeze', 5, 20)
    dropout_rate_ae = trial.suggest_float('dropout_rate_ae', 0.1, 0.5)
    dropout_rate_combined = trial.suggest_float('dropout_rate_combined', 0.1, 0.5)
    
    recon_weight = trial.suggest_float('recon_weight', 0.1, 0.9)
    class_weight = 1.0 - recon_weight
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingAutoencoder(X_train.shape[1], latent_dim, hidden_dim_ae, dropout_rate_ae)
        autoencoder = train_classifying_autoencoder(autoencoder, X_train, y_train, epochs_ae, batch_size, lr_ae, recon_weight, class_weight)
        
        combined_model = CombinedClassifier(autoencoder, X_pgs_train.shape[1], hidden_dim_combined, dropout_rate_combined)
        combined_model = train_combined_classifier(combined_model, X_train, X_pgs_train, y_train, epochs_freeze, epochs_unfreeze, batch_size, lr_combined)
        
        y_pred_proba = predict_combined(combined_model, X_val, X_pgs_val)
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
autoencoder = train_classifying_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs_ae'], 32, 
    best_params['lr_ae'], best_params['recon_weight'], 1.0 - best_params['recon_weight']
)

combined_model = CombinedClassifier(autoencoder, X_train_pgs.shape[1], best_params['hidden_dim_combined'], best_params['dropout_rate_combined'])
combined_model = train_combined_classifier(
    combined_model, X_train_all, X_train_pgs, y_all_train, 
    best_params['epochs_freeze'], best_params['epochs_unfreeze'], 32, best_params['lr_combined']
)

y_pred_proba = predict_combined(combined_model, X_val_all, X_val_pgs)
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 22:16:05,418] A new study created in memory with name: no-name-1937c797-2563-4cf4-bdc8-ffa00fdd5f6f


[I 2025-05-10 22:16:19,482] Trial 0 finished with value: 0.7630741507553102 and parameters: {'latent_dim': 39, 'hidden_dim_ae': 144, 'hidden_dim_combined': 33, 'lr_ae': 0.00022500979189260536, 'lr_combined': 0.0005796070741164595, 'epochs_ae': 15, 'epochs_freeze': 7, 'epochs_unfreeze': 18, 'dropout_rate_ae': 0.4916343137842111, 'dropout_rate_combined': 0.15687399844709649, 'recon_weight': 0.72013168762003}. Best is trial 0 with value: 0.7630741507553102.
[I 2025-05-10 22:16:34,776] Trial 1 finished with value: 0.6350356567747871 and parameters: {'latent_dim': 52, 'hidden_dim_ae': 182, 'hidden_dim_combined': 128, 'lr_ae': 0.003150179173782054, 'lr_combined': 0.00024094452968125498, 'epochs_ae': 28, 'epochs_freeze': 16, 'epochs_unfreeze': 7, 'dropout_rate_ae': 0.39756023309427324, 'dropout_rate_combined': 0.25855779849775384, 'recon_weight': 0.29944487925592245}. Best is trial 0 with value: 0.7630741507553102.
[I 2025-05-10 22:16:46,810] Trial 2 finished with value: 0.7431561996779387 an

ROC-AUC autoencoded bigset: 0.7650


In [74]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
        autoencoder = train_classifying_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs_ae'], 32, 
            best_params['lr_ae'], best_params['recon_weight'], 1.0 - best_params['recon_weight']
        )
        
        combined_model = CombinedClassifier(autoencoder, train_pgs.shape[1], best_params['hidden_dim_combined'], best_params['dropout_rate_combined'])
        combined_model = train_combined_classifier(
            combined_model, X_train, train_pgs, y_train, 
            best_params['epochs_freeze'], best_params['epochs_unfreeze'], 32, best_params['lr_combined']
        )
        
        y_pred_proba = predict_combined(combined_model, X_val, test_pgs)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7464
ROC-AUC autoencoded: 0.7405
ROC-AUC autoencoded: 0.7531
ROC-AUC autoencoded: 0.7479
ROC-AUC autoencoded: 0.7465
ROC-AUC autoencoded: 0.7647
ROC-AUC autoencoded: 0.7680
ROC-AUC autoencoded: 0.7794
ROC-AUC autoencoded: 0.7481
ROC-AUC autoencoded: 0.7682
ROC-AUC autoencoded: 0.7244
ROC-AUC autoencoded: 0.7250
ROC-AUC autoencoded: 0.7248
ROC-AUC autoencoded: 0.7316
ROC-AUC autoencoded: 0.7314
ROC-AUC autoencoded: 0.7028
ROC-AUC autoencoded: 0.6912
ROC-AUC autoencoded: 0.7136
ROC-AUC autoencoded: 0.7125
ROC-AUC autoencoded: 0.7048
ROC-AUC autoencoded: 0.7546
ROC-AUC autoencoded: 0.7781
ROC-AUC autoencoded: 0.7485
ROC-AUC autoencoded: 0.7558
ROC-AUC autoencoded: 0.7627
среднее 0.7409831682295444


Дальше идет блок со sparse автоэнкодером без классифицирующей головы

Логистическая регрессия

In [75]:
class SparseAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim=128, dropout_rate=0.2):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.LeakyReLU(0.2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, input_dim)
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent
    
def train_sparse_autoencoder(model, X_train, epochs, batch_size, lr, sparsity_weight):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, _ in train_loader:
            optimizer.zero_grad()
            reconstructed, latent = model(batch_x)
            
            recon_loss = criterion(reconstructed, batch_x)
            
            sparsity_loss = torch.mean(torch.abs(latent))
            
            loss = recon_loss + sparsity_weight * sparsity_loss
            
            loss.backward()
            optimizer.step()
    
    return model

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        _, latent = model(X_tensor)
    return latent.cpu().numpy()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    sparsity_weight = trial.suggest_float('sparsity_weight', 1e-4, 1e-1, log=True)
    
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'lbfgs'])
    
    rocs = []
    for i in range(3):
        autoencoder = SparseAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_sparse_autoencoder(autoencoder, X_train, epochs, batch_size, lr, sparsity_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        logreg = LogisticRegression(
            C=C,
            solver=solver,
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = SparseAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_sparse_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'], best_params['sparsity_weight'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

logreg = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=1000,
    random_state=42
)
logreg.fit(X_train_latent, y_all_train)

y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 22:24:29,551] A new study created in memory with name: no-name-53ed7951-e0a0-48aa-b241-b0f2db5b9f42
[I 2025-05-10 22:24:40,819] Trial 0 finished with value: 0.7609654167625183 and parameters: {'latent_dim': 32, 'hidden_dim': 193, 'lr': 0.000439054835052641, 'epochs': 37, 'dropout_rate': 0.26356971038867627, 'sparsity_weight': 0.0037130881811672936, 'C': 0.2273128169568188, 'solver': 'liblinear'}. Best is trial 0 with value: 0.7609654167625183.
[I 2025-05-10 22:24:46,217] Trial 1 finished with value: 0.7731577333026607 and parameters: {'latent_dim': 21, 'hidden_dim': 148, 'lr': 0.0002852222563059938, 'epochs': 20, 'dropout_rate': 0.14634224735361331, 'sparsity_weight': 0.06874570719718005, 'C': 0.38489667229645136, 'solver': 'liblinear'}. Best is trial 1 with value: 0.7731577333026607.
[I 2025-05-10 22:24:55,706] Trial 2 finished with value: 0.7632850241545893 and parameters: {'latent_dim': 75, 'hidden_dim': 94, 'lr': 0.00043951678703255435, 'epochs': 37, 'dropout_rate': 0

ROC-AUC autoencoded bigset: 0.7768


In [76]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = SparseAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_sparse_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'], best_params['sparsity_weight'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        logreg = LogisticRegression(
            C=best_params['C'],
            solver=best_params['solver'],
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7551
ROC-AUC autoencoded: 0.7581
ROC-AUC autoencoded: 0.7639
ROC-AUC autoencoded: 0.7451
ROC-AUC autoencoded: 0.7541
ROC-AUC autoencoded: 0.7614
ROC-AUC autoencoded: 0.7625
ROC-AUC autoencoded: 0.7762
ROC-AUC autoencoded: 0.7789
ROC-AUC autoencoded: 0.7806
ROC-AUC autoencoded: 0.7348
ROC-AUC autoencoded: 0.7506
ROC-AUC autoencoded: 0.7358
ROC-AUC autoencoded: 0.7495
ROC-AUC autoencoded: 0.7419
ROC-AUC autoencoded: 0.7125
ROC-AUC autoencoded: 0.7123
ROC-AUC autoencoded: 0.7104
ROC-AUC autoencoded: 0.7003
ROC-AUC autoencoded: 0.7104
ROC-AUC autoencoded: 0.7577
ROC-AUC autoencoded: 0.7610
ROC-AUC autoencoded: 0.7688
ROC-AUC autoencoded: 0.7509
ROC-AUC autoencoded: 0.7609
среднее 0.7477580528475479


Градиентный бустинг

In [77]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    sparsity_weight = trial.suggest_float('sparsity_weight', 1e-4, 1e-1, log=True)
    
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    rocs = []
    for i in range(3):
        autoencoder = SparseAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_sparse_autoencoder(autoencoder, X_train, epochs, batch_size, lr, sparsity_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = SparseAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_sparse_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'], best_params['sparsity_weight'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

gb = GradientBoostingClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
gb.fit(X_train_latent, y_all_train)

y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 22:28:54,784] A new study created in memory with name: no-name-7414702f-52c0-4757-9709-0910321f91f8


[I 2025-05-10 22:30:31,745] Trial 0 finished with value: 0.7316156736446592 and parameters: {'latent_dim': 26, 'hidden_dim': 64, 'lr': 0.0006469295284087385, 'epochs': 24, 'dropout_rate': 0.44588186205422464, 'sparsity_weight': 0.0023930896689645987, 'n_estimators': 425, 'max_depth': 6, 'learning_rate': 0.041580179378465205, 'subsample': 0.8300983689591996, 'min_samples_split': 8, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7316156736446592.
[I 2025-05-10 22:31:47,227] Trial 1 finished with value: 0.7169120466221915 and parameters: {'latent_dim': 28, 'hidden_dim': 256, 'lr': 0.006038845936678987, 'epochs': 48, 'dropout_rate': 0.18524211155355952, 'sparsity_weight': 0.02546856117329749, 'n_estimators': 293, 'max_depth': 5, 'learning_rate': 0.16778073282756392, 'subsample': 0.9541033040936405, 'min_samples_split': 16, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.7316156736446592.
[I 2025-05-10 22:32:20,498] Trial 2 finished with value: 0.7447473353270455 and parameters

ROC-AUC autoencoded bigset: 0.7588


In [78]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = SparseAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_sparse_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'], best_params['sparsity_weight'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            subsample=best_params['subsample'],
            min_samples_split=best_params['min_samples_split'],
            min_samples_leaf=best_params['min_samples_leaf'],
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7501
ROC-AUC autoencoded: 0.7396
ROC-AUC autoencoded: 0.7440
ROC-AUC autoencoded: 0.7371
ROC-AUC autoencoded: 0.7356
ROC-AUC autoencoded: 0.7367
ROC-AUC autoencoded: 0.7349
ROC-AUC autoencoded: 0.7368
ROC-AUC autoencoded: 0.7281
ROC-AUC autoencoded: 0.7318
ROC-AUC autoencoded: 0.7510
ROC-AUC autoencoded: 0.7338
ROC-AUC autoencoded: 0.7361
ROC-AUC autoencoded: 0.7333
ROC-AUC autoencoded: 0.7384
ROC-AUC autoencoded: 0.7252
ROC-AUC autoencoded: 0.7331
ROC-AUC autoencoded: 0.7209
ROC-AUC autoencoded: 0.7206
ROC-AUC autoencoded: 0.7254
ROC-AUC autoencoded: 0.7582
ROC-AUC autoencoded: 0.7544
ROC-AUC autoencoded: 0.7560
ROC-AUC autoencoded: 0.7618
ROC-AUC autoencoded: 0.7588
среднее 0.7392669053725274


SVC

In [79]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    sparsity_weight = trial.suggest_float('sparsity_weight', 1e-4, 1e-1, log=True)
    
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    
    rocs = []
    for i in range(3):
        autoencoder = SparseAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_sparse_autoencoder(autoencoder, X_train, epochs, batch_size, lr, sparsity_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        svc = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            degree=degree,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = SparseAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_sparse_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'], best_params['sparsity_weight'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

svc = SVC(
    C=best_params['C'],
    kernel=best_params['kernel'],
    gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
    degree=best_params['degree'] if 'degree' in best_params else 3,
    probability=True,
    random_state=42
)
svc.fit(X_train_latent, y_all_train)

y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 22:51:33,176] A new study created in memory with name: no-name-f83d3b13-7cfc-4b47-9bdc-bb9a4410ba41
[I 2025-05-10 22:51:43,557] Trial 0 finished with value: 0.6364542596426654 and parameters: {'latent_dim': 66, 'hidden_dim': 163, 'lr': 0.006176910533801481, 'epochs': 21, 'dropout_rate': 0.4414074977951513, 'sparsity_weight': 0.005262446367759267, 'C': 1.0649836963445984, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 0 with value: 0.6364542596426654.
[I 2025-05-10 22:51:54,819] Trial 1 finished with value: 0.7312130971551262 and parameters: {'latent_dim': 56, 'hidden_dim': 200, 'lr': 0.00019401128345167122, 'epochs': 24, 'dropout_rate': 0.18963744770433127, 'sparsity_weight': 0.0010726501375091877, 'C': 0.25770499698101157, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 1 with value: 0.7312130971551262.
[I 2025-05-10 22:51:59,678] Trial 2 finished with value: 0.6630626485698948 and parameters: {'latent_dim': 15, 'hidden_dim': 98, 'lr': 0.007511135867907253, 'epo

ROC-AUC autoencoded bigset: 0.7576


In [80]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = SparseAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_sparse_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'], best_params['sparsity_weight'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        svc = SVC(
            C=best_params['C'],
            kernel=best_params['kernel'],
            gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
            degree=best_params['degree'] if 'degree' in best_params else 3,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7234
ROC-AUC autoencoded: 0.7188
ROC-AUC autoencoded: 0.7034
ROC-AUC autoencoded: 0.7399
ROC-AUC autoencoded: 0.7131
ROC-AUC autoencoded: 0.7587
ROC-AUC autoencoded: 0.7331
ROC-AUC autoencoded: 0.7506
ROC-AUC autoencoded: 0.7545
ROC-AUC autoencoded: 0.7273
ROC-AUC autoencoded: 0.7297
ROC-AUC autoencoded: 0.7037
ROC-AUC autoencoded: 0.7378
ROC-AUC autoencoded: 0.7366
ROC-AUC autoencoded: 0.7437
ROC-AUC autoencoded: 0.6256
ROC-AUC autoencoded: 0.6859
ROC-AUC autoencoded: 0.6161
ROC-AUC autoencoded: 0.6450
ROC-AUC autoencoded: 0.6193
ROC-AUC autoencoded: 0.7341
ROC-AUC autoencoded: 0.7149
ROC-AUC autoencoded: 0.7087
ROC-AUC autoencoded: 0.7332
ROC-AUC autoencoded: 0.7291
среднее 0.7114574902519419


MLP

In [81]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate=0.2):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

def train_mlp(model, X_train, y_train, epochs, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    return model

def predict_mlp(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        predictions = model(X_tensor)
    
    return predictions.cpu().numpy().flatten()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim_ae = trial.suggest_int('hidden_dim_ae', 64, 256)
    lr_ae = trial.suggest_float('lr_ae', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs_ae = trial.suggest_int('epochs_ae', 10, 50)
    dropout_rate_ae = trial.suggest_float('dropout_rate_ae', 0.1, 0.5)
    
    sparsity_weight = trial.suggest_float('sparsity_weight', 1e-4, 1e-1, log=True)
    
    hidden_dim_mlp = trial.suggest_int('hidden_dim_mlp', 32, 128)
    lr_mlp = trial.suggest_float('lr_mlp', 1e-4, 1e-2, log=True)
    epochs_mlp = trial.suggest_int('epochs_mlp', 10, 50)
    dropout_rate_mlp = trial.suggest_float('dropout_rate_mlp', 0.1, 0.5)
    
    rocs = []
    for i in range(3):
        autoencoder = SparseAutoencoder(X_train.shape[1], latent_dim, hidden_dim_ae, dropout_rate_ae)
        autoencoder = train_sparse_autoencoder(autoencoder, X_train, epochs_ae, batch_size, lr_ae, sparsity_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        mlp = MLP(X_train_latent.shape[1], hidden_dim_mlp, dropout_rate_mlp)
        mlp = train_mlp(mlp, X_train_latent, y_train, epochs_mlp, batch_size, lr_mlp)
        
        y_pred_proba = predict_mlp(mlp, X_val_latent)
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = SparseAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
autoencoder = train_sparse_autoencoder(autoencoder, X_train_all, best_params['epochs_ae'], 32, best_params['lr_ae'], best_params['sparsity_weight'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

mlp = MLP(X_train_latent.shape[1], best_params['hidden_dim_mlp'], best_params['dropout_rate_mlp'])
mlp = train_mlp(mlp, X_train_latent, y_all_train, best_params['epochs_mlp'], 32, best_params['lr_mlp'])

y_pred_proba = predict_mlp(mlp, X_val_latent)
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 23:18:45,446] A new study created in memory with name: no-name-b8ba6cbe-46ee-4cf1-81d7-b8eefd91c8d1
[I 2025-05-10 23:19:03,108] Trial 0 finished with value: 0.6973966720343533 and parameters: {'latent_dim': 90, 'hidden_dim_ae': 120, 'lr_ae': 0.00037056821636439563, 'epochs_ae': 43, 'dropout_rate_ae': 0.24079394383062028, 'sparsity_weight': 0.009851395724032133, 'hidden_dim_mlp': 72, 'lr_mlp': 0.0015644814119993757, 'epochs_mlp': 42, 'dropout_rate_mlp': 0.36742416059859173}. Best is trial 0 with value: 0.6973966720343533.
[I 2025-05-10 23:19:16,952] Trial 1 finished with value: 0.7408365922858676 and parameters: {'latent_dim': 61, 'hidden_dim_ae': 95, 'lr_ae': 0.0019619413006113705, 'epochs_ae': 41, 'dropout_rate_ae': 0.18593520624481258, 'sparsity_weight': 0.0001204419304255439, 'hidden_dim_mlp': 119, 'lr_mlp': 0.0007261093340923912, 'epochs_mlp': 23, 'dropout_rate_mlp': 0.48146076073704247}. Best is trial 1 with value: 0.7408365922858676.
[I 2025-05-10 23:19:30,139] Tria

ROC-AUC autoencoded bigset: 0.7608


In [82]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = SparseAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
        autoencoder = train_sparse_autoencoder(autoencoder, X_train, best_params['epochs_ae'], 32, best_params['lr_ae'], best_params['sparsity_weight'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        mlp = MLP(X_train_latent.shape[1], best_params['hidden_dim_mlp'], best_params['dropout_rate_mlp'])
        mlp = train_mlp(mlp, X_train_latent, y_train, best_params['epochs_mlp'], 32, best_params['lr_mlp'])
        
        y_pred_proba = predict_mlp(mlp, X_val_latent)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7454
ROC-AUC autoencoded: 0.7526
ROC-AUC autoencoded: 0.7426
ROC-AUC autoencoded: 0.7205
ROC-AUC autoencoded: 0.7285
ROC-AUC autoencoded: 0.7619
ROC-AUC autoencoded: 0.7546
ROC-AUC autoencoded: 0.7447
ROC-AUC autoencoded: 0.7493
ROC-AUC autoencoded: 0.7498
ROC-AUC autoencoded: 0.7032
ROC-AUC autoencoded: 0.7394
ROC-AUC autoencoded: 0.7160
ROC-AUC autoencoded: 0.7119
ROC-AUC autoencoded: 0.7104
ROC-AUC autoencoded: 0.7120
ROC-AUC autoencoded: 0.6804
ROC-AUC autoencoded: 0.6664
ROC-AUC autoencoded: 0.7199
ROC-AUC autoencoded: 0.7087
ROC-AUC autoencoded: 0.7523
ROC-AUC autoencoded: 0.7495
ROC-AUC autoencoded: 0.7510
ROC-AUC autoencoded: 0.7283
ROC-AUC autoencoded: 0.7603
среднее 0.7303867485098837


Теперь блок с sparse автоэнкодером с классифицирующей головой

Логистическая регрессия

In [83]:
class ClassifyingSparseAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim=128, dropout_rate=0.2):
        super(ClassifyingSparseAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.LeakyReLU(0.2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, input_dim)
        )
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        classification = self.classifier(latent)
        return reconstructed, latent, classification
    
def train_classifying_sparse_autoencoder(model, X_train, y_train, epochs, batch_size, lr, sparsity_weight, classification_weight):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    recon_criterion = nn.MSELoss()
    class_criterion = nn.BCELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            reconstructed, latent, classification = model(batch_x)
            
            recon_loss = recon_criterion(reconstructed, batch_x)
            sparsity_loss = torch.mean(torch.abs(latent))
            class_loss = class_criterion(classification, batch_y)
            
            loss = (1 - classification_weight) * (recon_loss + sparsity_weight * sparsity_loss) + classification_weight * class_loss
            
            loss.backward()
            optimizer.step()
    
    return model

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        _, latent, _ = model(X_tensor)
    return latent.cpu().numpy()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    sparsity_weight = trial.suggest_float('sparsity_weight', 1e-4, 1e-1, log=True)
    classification_weight = trial.suggest_float('classification_weight', 0.1, 0.9)
    
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'lbfgs'])
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingSparseAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_classifying_sparse_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, sparsity_weight, classification_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        logreg = LogisticRegression(
            C=C,
            solver=solver,
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingSparseAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_classifying_sparse_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, 
    best_params['lr'], best_params['sparsity_weight'], best_params['classification_weight']
)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

logreg = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=1000,
    random_state=42
)
logreg.fit(X_train_latent, y_all_train)

y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 23:24:39,640] A new study created in memory with name: no-name-287d9212-e000-45f1-bbb8-21094037c7ca
[I 2025-05-10 23:24:48,103] Trial 0 finished with value: 0.6132390154129284 and parameters: {'latent_dim': 87, 'hidden_dim': 95, 'lr': 0.0002597725135296698, 'epochs': 26, 'dropout_rate': 0.17445431622961638, 'sparsity_weight': 0.0076835035396832325, 'classification_weight': 0.4300906142523754, 'C': 4.176997941941316, 'solver': 'liblinear'}. Best is trial 0 with value: 0.6132390154129284.
[I 2025-05-10 23:25:04,215] Trial 1 finished with value: 0.5690131124913733 and parameters: {'latent_dim': 29, 'hidden_dim': 251, 'lr': 0.003097635834286806, 'epochs': 46, 'dropout_rate': 0.40385626626354487, 'sparsity_weight': 0.0005577221789078868, 'classification_weight': 0.44866056643364016, 'C': 0.6197177320180327, 'solver': 'liblinear'}. Best is trial 0 with value: 0.6132390154129284.
[I 2025-05-10 23:25:08,490] Trial 2 finished with value: 0.7171804309485469 and parameters: {'latent

ROC-AUC autoencoded bigset: 0.7705


In [84]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingSparseAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_classifying_sparse_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs'], 32, 
            best_params['lr'], best_params['sparsity_weight'], best_params['classification_weight']
        )
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        logreg = LogisticRegression(
            C=best_params['C'],
            solver=best_params['solver'],
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7302
ROC-AUC autoencoded: 0.7115
ROC-AUC autoencoded: 0.7241
ROC-AUC autoencoded: 0.7283
ROC-AUC autoencoded: 0.7264
ROC-AUC autoencoded: 0.7726
ROC-AUC autoencoded: 0.7745
ROC-AUC autoencoded: 0.7731
ROC-AUC autoencoded: 0.7750
ROC-AUC autoencoded: 0.7720
ROC-AUC autoencoded: 0.7445
ROC-AUC autoencoded: 0.7419
ROC-AUC autoencoded: 0.7413
ROC-AUC autoencoded: 0.7359
ROC-AUC autoencoded: 0.7371
ROC-AUC autoencoded: 0.6875
ROC-AUC autoencoded: 0.6855
ROC-AUC autoencoded: 0.6841
ROC-AUC autoencoded: 0.6787
ROC-AUC autoencoded: 0.6730
ROC-AUC autoencoded: 0.7548
ROC-AUC autoencoded: 0.7477
ROC-AUC autoencoded: 0.7511
ROC-AUC autoencoded: 0.7499
ROC-AUC autoencoded: 0.7468
среднее 0.7339096574359116


градиентный бустинг

In [85]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    sparsity_weight = trial.suggest_float('sparsity_weight', 1e-4, 1e-1, log=True)
    classification_weight = trial.suggest_float('classification_weight', 0.1, 0.9)
    
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingSparseAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_classifying_sparse_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, sparsity_weight, classification_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingSparseAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_classifying_sparse_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, 
    best_params['lr'], best_params['sparsity_weight'], best_params['classification_weight']
)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

gb = GradientBoostingClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
gb.fit(X_train_latent, y_all_train)

y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 23:30:20,924] A new study created in memory with name: no-name-679de66c-a083-415a-8b32-fd308ce86be3
[I 2025-05-10 23:31:05,661] Trial 0 finished with value: 0.639751552795031 and parameters: {'latent_dim': 44, 'hidden_dim': 98, 'lr': 0.0001515986697605629, 'epochs': 30, 'dropout_rate': 0.10648123718712915, 'sparsity_weight': 0.09779898429249859, 'classification_weight': 0.8332143125303054, 'n_estimators': 180, 'max_depth': 9, 'learning_rate': 0.06646491611984465, 'subsample': 0.6605850657689435, 'min_samples_split': 17, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.639751552795031.
[I 2025-05-10 23:33:03,609] Trial 1 finished with value: 0.6335978835978836 and parameters: {'latent_dim': 31, 'hidden_dim': 169, 'lr': 0.0008349457937271405, 'epochs': 13, 'dropout_rate': 0.28243490475826083, 'sparsity_weight': 0.00014538510913989083, 'classification_weight': 0.89467242270219, 'n_estimators': 487, 'max_depth': 8, 'learning_rate': 0.012565844169546684, 'subsample': 0.90

ROC-AUC autoencoded bigset: 0.7146


In [86]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingSparseAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_classifying_sparse_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs'], 32, 
            best_params['lr'], best_params['sparsity_weight'], best_params['classification_weight']
        )
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            subsample=best_params['subsample'],
            min_samples_split=best_params['min_samples_split'],
            min_samples_leaf=best_params['min_samples_leaf'],
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7249
ROC-AUC autoencoded: 0.7070
ROC-AUC autoencoded: 0.7117
ROC-AUC autoencoded: 0.6903
ROC-AUC autoencoded: 0.7191
ROC-AUC autoencoded: 0.7238
ROC-AUC autoencoded: 0.7080
ROC-AUC autoencoded: 0.7098
ROC-AUC autoencoded: 0.7051
ROC-AUC autoencoded: 0.6939
ROC-AUC autoencoded: 0.7443
ROC-AUC autoencoded: 0.7035
ROC-AUC autoencoded: 0.7183
ROC-AUC autoencoded: 0.6877
ROC-AUC autoencoded: 0.7429
ROC-AUC autoencoded: 0.7085
ROC-AUC autoencoded: 0.7017
ROC-AUC autoencoded: 0.7004
ROC-AUC autoencoded: 0.6905
ROC-AUC autoencoded: 0.7054
ROC-AUC autoencoded: 0.7334
ROC-AUC autoencoded: 0.7311
ROC-AUC autoencoded: 0.7516
ROC-AUC autoencoded: 0.7309
ROC-AUC autoencoded: 0.7651
среднее 0.7163534865444702


SVC

In [87]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    sparsity_weight = trial.suggest_float('sparsity_weight', 1e-4, 1e-1, log=True)
    classification_weight = trial.suggest_float('classification_weight', 0.1, 0.9)
    
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingSparseAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_classifying_sparse_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, sparsity_weight, classification_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        svc = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            degree=degree,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingSparseAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_classifying_sparse_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, 
    best_params['lr'], best_params['sparsity_weight'], best_params['classification_weight']
)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

svc = SVC(
    C=best_params['C'],
    kernel=best_params['kernel'],
    gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
    degree=best_params['degree'] if 'degree' in best_params else 3,
    probability=True,
    random_state=42
)
svc.fit(X_train_latent, y_all_train)

y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-10 23:57:58,409] A new study created in memory with name: no-name-ead7a59b-6290-42b9-8426-abbe628df1eb
[I 2025-05-10 23:58:12,327] Trial 0 finished with value: 0.6499693275055595 and parameters: {'latent_dim': 10, 'hidden_dim': 150, 'lr': 0.003181444268378388, 'epochs': 41, 'dropout_rate': 0.30293301483924245, 'sparsity_weight': 0.00939408379252416, 'classification_weight': 0.42908216702915325, 'C': 0.404588340373358, 'kernel': 'poly', 'gamma': 'auto', 'degree': 2}. Best is trial 0 with value: 0.6499693275055595.
[I 2025-05-10 23:58:17,957] Trial 1 finished with value: 0.6811785905988804 and parameters: {'latent_dim': 42, 'hidden_dim': 117, 'lr': 0.00013081847557917228, 'epochs': 15, 'dropout_rate': 0.152885745608848, 'sparsity_weight': 0.019224790719018805, 'classification_weight': 0.729320197473523, 'C': 0.3233723985191251, 'kernel': 'poly', 'gamma': 'scale', 'degree': 5}. Best is trial 1 with value: 0.6811785905988804.
[I 2025-05-10 23:58:31,367] Trial 2 finished with val

ROC-AUC autoencoded bigset: 0.7440


In [88]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingSparseAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_classifying_sparse_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs'], 32, 
            best_params['lr'], best_params['sparsity_weight'], best_params['classification_weight']
        )
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        svc = SVC(
            C=best_params['C'],
            kernel=best_params['kernel'],
            gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
            degree=best_params['degree'] if 'degree' in best_params else 3,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7179
ROC-AUC autoencoded: 0.7078
ROC-AUC autoencoded: 0.7166
ROC-AUC autoencoded: 0.7026
ROC-AUC autoencoded: 0.7164
ROC-AUC autoencoded: 0.7244
ROC-AUC autoencoded: 0.7330
ROC-AUC autoencoded: 0.7306
ROC-AUC autoencoded: 0.7338
ROC-AUC autoencoded: 0.7512
ROC-AUC autoencoded: 0.7266
ROC-AUC autoencoded: 0.6988
ROC-AUC autoencoded: 0.7290
ROC-AUC autoencoded: 0.7000
ROC-AUC autoencoded: 0.7230
ROC-AUC autoencoded: 0.7092
ROC-AUC autoencoded: 0.6915
ROC-AUC autoencoded: 0.6962
ROC-AUC autoencoded: 0.6967
ROC-AUC autoencoded: 0.6944
ROC-AUC autoencoded: 0.7599
ROC-AUC autoencoded: 0.7543
ROC-AUC autoencoded: 0.7636
ROC-AUC autoencoded: 0.7565
ROC-AUC autoencoded: 0.7663
среднее 0.7240005454317098


MLP

In [89]:
class CombinedClassifier(nn.Module):
    def __init__(self, autoencoder, pgs_input_dim, hidden_dim=64, dropout_rate=0.2):
        super(CombinedClassifier, self).__init__()
        self.autoencoder = autoencoder
        self.pgs_branch = nn.Sequential(
            nn.Linear(pgs_input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2)
        )
        latent_dim = list(autoencoder.classifier[0].parameters())[0].shape[1]
        self.combined_classifier = nn.Sequential(
            nn.Linear(hidden_dim // 2 + latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x_snp, x_pgs):
        _, latent, _ = self.autoencoder(x_snp)
        pgs_features = self.pgs_branch(x_pgs)
        combined = torch.cat([latent, pgs_features], dim=1)
        output = self.combined_classifier(combined)
        return output

def train_combined_classifier(model, X_train_snp, X_train_pgs, y_train, epochs_freeze, epochs_unfreeze, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    X_train_snp_tensor = torch.FloatTensor(X_train_snp).to(device)
    X_train_pgs_tensor = torch.FloatTensor(X_train_pgs).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_snp_tensor, X_train_pgs_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    criterion = nn.BCELoss()
    
    for param in model.autoencoder.parameters():
        param.requires_grad = False
    
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    model.train()
    for epoch in range(epochs_freeze):
        for batch_snp, batch_pgs, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_snp, batch_pgs)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    for param in model.autoencoder.parameters():
        param.requires_grad = True
    
    optimizer = optim.Adam(model.parameters(), lr=lr/10)
    
    for epoch in range(epochs_unfreeze):
        for batch_snp, batch_pgs, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_snp, batch_pgs)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    return model

def predict_combined(model, X_snp, X_pgs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    X_snp_tensor = torch.FloatTensor(X_snp).to(device)
    X_pgs_tensor = torch.FloatTensor(X_pgs).to(device)
    
    with torch.no_grad():
        predictions = model(X_snp_tensor, X_pgs_tensor)
    
    return predictions.cpu().numpy().flatten()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim_ae = trial.suggest_int('hidden_dim_ae', 64, 256)
    hidden_dim_combined = trial.suggest_int('hidden_dim_combined', 32, 128)
    lr_ae = trial.suggest_float('lr_ae', 1e-4, 1e-2, log=True)
    lr_combined = trial.suggest_float('lr_combined', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs_ae = trial.suggest_int('epochs_ae', 10, 50)
    epochs_freeze = trial.suggest_int('epochs_freeze', 5, 20)
    epochs_unfreeze = trial.suggest_int('epochs_unfreeze', 5, 20)
    dropout_rate_ae = trial.suggest_float('dropout_rate_ae', 0.1, 0.5)
    dropout_rate_combined = trial.suggest_float('dropout_rate_combined', 0.1, 0.5)
    
    sparsity_weight = trial.suggest_float('sparsity_weight', 1e-4, 1e-1, log=True)
    classification_weight = trial.suggest_float('classification_weight', 0.1, 0.9)
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingSparseAutoencoder(X_train.shape[1], latent_dim, hidden_dim_ae, dropout_rate_ae)
        autoencoder = train_classifying_sparse_autoencoder(autoencoder, X_train, y_train, epochs_ae, batch_size, lr_ae, sparsity_weight, classification_weight)
        
        combined_model = CombinedClassifier(autoencoder, X_pgs_train.shape[1], hidden_dim_combined, dropout_rate_combined)
        combined_model = train_combined_classifier(combined_model, X_train, X_pgs_train, y_train, epochs_freeze, epochs_unfreeze, batch_size, lr_combined)
        
        y_pred_proba = predict_combined(combined_model, X_val, X_pgs_val)
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingSparseAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
autoencoder = train_classifying_sparse_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs_ae'], 32, 
    best_params['lr_ae'], best_params['sparsity_weight'], best_params['classification_weight']
)

combined_model = CombinedClassifier(autoencoder, X_train_pgs.shape[1], best_params['hidden_dim_combined'], best_params['dropout_rate_combined'])
combined_model = train_combined_classifier(
    combined_model, X_train_all, X_train_pgs, y_all_train, 
    best_params['epochs_freeze'], best_params['epochs_unfreeze'], 32, best_params['lr_combined']
)

y_pred_proba = predict_combined(combined_model, X_val_all, X_val_pgs)
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 00:03:01,151] A new study created in memory with name: no-name-957c9cfb-363f-45a8-aa04-2882ba782f78
[I 2025-05-11 00:03:23,943] Trial 0 finished with value: 0.7486389080591979 and parameters: {'latent_dim': 12, 'hidden_dim_ae': 135, 'hidden_dim_combined': 53, 'lr_ae': 0.00038001124498365877, 'lr_combined': 0.0002619789947568248, 'epochs_ae': 47, 'epochs_freeze': 6, 'epochs_unfreeze': 18, 'dropout_rate_ae': 0.13029327245332567, 'dropout_rate_combined': 0.12058091298621215, 'sparsity_weight': 0.011840287681188794, 'classification_weight': 0.13791395931112957}. Best is trial 0 with value: 0.7486389080591979.
[I 2025-05-11 00:03:46,028] Trial 1 finished with value: 0.5856146001073537 and parameters: {'latent_dim': 49, 'hidden_dim_ae': 155, 'hidden_dim_combined': 103, 'lr_ae': 0.005009279122342667, 'lr_combined': 0.0019115482008657005, 'epochs_ae': 41, 'epochs_freeze': 14, 'epochs_unfreeze': 14, 'dropout_rate_ae': 0.20375740690750682, 'dropout_rate_combined': 0.248281690791986

ROC-AUC autoencoded bigset: 0.7694


In [90]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingSparseAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
        autoencoder = train_classifying_sparse_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs_ae'], 32, 
            best_params['lr_ae'], best_params['sparsity_weight'], best_params['classification_weight']
        )
        
        combined_model = CombinedClassifier(autoencoder, train_pgs.shape[1], best_params['hidden_dim_combined'], best_params['dropout_rate_combined'])
        combined_model = train_combined_classifier(
            combined_model, X_train, train_pgs, y_train, 
            best_params['epochs_freeze'], best_params['epochs_unfreeze'], 32, best_params['lr_combined']
        )
        
        y_pred_proba = predict_combined(combined_model, X_val, test_pgs)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7254
ROC-AUC autoencoded: 0.7293
ROC-AUC autoencoded: 0.7268
ROC-AUC autoencoded: 0.7105
ROC-AUC autoencoded: 0.7271
ROC-AUC autoencoded: 0.7738
ROC-AUC autoencoded: 0.7677
ROC-AUC autoencoded: 0.7621
ROC-AUC autoencoded: 0.7459
ROC-AUC autoencoded: 0.7707
ROC-AUC autoencoded: 0.7297
ROC-AUC autoencoded: 0.7188
ROC-AUC autoencoded: 0.7151
ROC-AUC autoencoded: 0.7454
ROC-AUC autoencoded: 0.7164
ROC-AUC autoencoded: 0.6762
ROC-AUC autoencoded: 0.6787
ROC-AUC autoencoded: 0.6770
ROC-AUC autoencoded: 0.6831
ROC-AUC autoencoded: 0.6817
ROC-AUC autoencoded: 0.7690
ROC-AUC autoencoded: 0.7577
ROC-AUC autoencoded: 0.7567
ROC-AUC autoencoded: 0.7537
ROC-AUC autoencoded: 0.7568
среднее 0.7302164715951672


Дальше идет блок со stacked автоэнкодером

Логистическая регрессия

In [91]:
class SimpleAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.2):
        super(SimpleAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate)
        )
        self.decoder = nn.Sequential(
            nn.Linear(output_dim, input_dim),
            nn.BatchNorm1d(input_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate)
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent

class StackedAutoencoder(nn.Module):
    def __init__(self, input_dim, layer_dims, dropout_rate=0.2):
        super(StackedAutoencoder, self).__init__()
        self.autoencoders = nn.ModuleList()
        
        # Создаем последовательность автоэнкодеров
        prev_dim = input_dim
        for dim in layer_dims:
            self.autoencoders.append(SimpleAutoencoder(prev_dim, dim, dropout_rate))
            prev_dim = dim
        
    def forward(self, x):
        latent = x
        reconstructions = []
        latents = []
        
        # Прямой проход через все автоэнкодеры
        for ae in self.autoencoders:
            reconstructed, latent = ae(latent)
            reconstructions.append(reconstructed)
            latents.append(latent)
        
        return reconstructions, latents
    
def train_stacked_autoencoder(model, X_train, epochs, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    
    # Обучаем каждый автоэнкодер последовательно
    latent = X_train_tensor
    for i, ae in enumerate(model.autoencoders):
        print(f"Training autoencoder {i+1}/{len(model.autoencoders)}")
        
        train_dataset = TensorDataset(latent, latent)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        optimizer = optim.Adam(ae.parameters(), lr=lr)
        criterion = nn.MSELoss()
        
        ae.train()
        for epoch in range(epochs):
            for batch_x, _ in train_loader:
                optimizer.zero_grad()
                reconstructed, new_latent = ae(batch_x)
                loss = criterion(reconstructed, batch_x)
                loss.backward()
                optimizer.step()
        
        # Получаем латентное представление для следующего автоэнкодера
        ae.eval()
        with torch.no_grad():
            _, latent = ae(latent)
    
    return model

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    
    # Получаем латентное представление из последнего автоэнкодера
    latent = X_tensor
    for ae in model.autoencoders:
        with torch.no_grad():
            _, latent = ae(latent)
            
    return latent.cpu().numpy()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    # Параметры stacked автоэнкодера
    num_layers = trial.suggest_int('num_layers', 2, 2)
    latent_dims = []
    
    # Определяем размерности слоев автоэнкодера
    dim = X_train.shape[1]
    for i in range(num_layers):
        dim = dim // 2
        dim = max(10, dim)  # Минимальный размер 10
        latent_dim = trial.suggest_int(f'latent_dim_{i}', max(8, dim//2), dim)
        latent_dims.append(latent_dim)
    
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    # Параметры логистической регрессии
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'lbfgs'])
    
    rocs = []
    for i in range(3):
        autoencoder = StackedAutoencoder(X_train.shape[1], latent_dims, dropout_rate)
        autoencoder = train_stacked_autoencoder(autoencoder, X_train, epochs, batch_size, lr)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        logreg = LogisticRegression(
            C=C,
            solver=solver,
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params

# Восстанавливаем список размерностей слоев из best_params
latent_dims = []
for i in range(best_params['num_layers']):
    latent_dims.append(best_params[f'latent_dim_{i}'])

autoencoder = StackedAutoencoder(X_train_all.shape[1], latent_dims, best_params['dropout_rate'])
autoencoder = train_stacked_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

logreg = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=1000,
    random_state=42
)
logreg.fit(X_train_latent, y_all_train)

y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 00:11:51,620] A new study created in memory with name: no-name-56608c85-5108-4f59-8e50-0deca92c48c4


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:12:00,148] Trial 0 finished with value: 0.7814201364926002 and parameters: {'num_layers': 2, 'latent_dim_0': 24, 'latent_dim_1': 17, 'lr': 0.005329257156731006, 'epochs': 23, 'dropout_rate': 0.2535867785704592, 'C': 0.01559747270245342, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.7814201364926002.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:12:06,364] Trial 1 finished with value: 0.767732535848478 and parameters: {'num_layers': 2, 'latent_dim_0': 22, 'latent_dim_1': 20, 'lr': 0.00025312136214161116, 'epochs': 18, 'dropout_rate': 0.4474397704329506, 'C': 4.743829355895827, 'solver': 'liblinear'}. Best is trial 0 with value: 0.7814201364926002.


...

Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:15:04,257] Trial 18 finished with value: 0.7777010965416763 and parameters: {'num_layers': 2, 'latent_dim_0': 37, 'latent_dim_1': 18, 'lr': 0.0017587902751428535, 'epochs': 10, 'dropout_rate': 0.3975879395503068, 'C': 0.023493764075224792, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.7814201364926002.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:15:16,349] Trial 19 finished with value: 0.7756498734759605 and parameters: {'num_layers': 2, 'latent_dim_0': 29, 'latent_dim_1': 16, 'lr': 0.004613709705449361, 'epochs': 48, 'dropout_rate': 0.40589093231491846, 'C': 0.12811734670255281, 'solver': 'liblinear'}. Best is trial 0 with value: 0.7814201364926002.


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded bigset: 0.7743


In [92]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        # Восстанавливаем список размерностей слоев из best_params
        latent_dims = []
        for k in range(best_params['num_layers']):
            latent_dims.append(best_params[f'latent_dim_{k}'])
            
        autoencoder = StackedAutoencoder(X_train.shape[1], latent_dims, best_params['dropout_rate'])
        autoencoder = train_stacked_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        logreg = LogisticRegression(
            C=best_params['C'],
            solver=best_params['solver'],
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7634
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7571
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7598
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7598
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7576
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7782
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7776
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7770
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7774
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7766
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7389
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7336
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoen

градиентный бустинг

In [93]:
class SimpleAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.2):
        super(SimpleAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate)
        )
        self.decoder = nn.Sequential(
            nn.Linear(output_dim, input_dim),
            nn.BatchNorm1d(input_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate)
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent

class StackedAutoencoder(nn.Module):
    def __init__(self, input_dim, layer_dims, dropout_rate=0.2):
        super(StackedAutoencoder, self).__init__()
        self.autoencoders = nn.ModuleList()
        
        # Создаем последовательность автоэнкодеров
        prev_dim = input_dim
        for dim in layer_dims:
            self.autoencoders.append(SimpleAutoencoder(prev_dim, dim, dropout_rate))
            prev_dim = dim
        
    def forward(self, x):
        latent = x
        reconstructions = []
        latents = []
        
        # Прямой проход через все автоэнкодеры
        for ae in self.autoencoders:
            reconstructed, latent = ae(latent)
            reconstructions.append(reconstructed)
            latents.append(latent)
        
        return reconstructions, latents
    
def train_stacked_autoencoder(model, X_train, epochs, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    
    # Обучаем каждый автоэнкодер последовательно
    latent = X_train_tensor
    for i, ae in enumerate(model.autoencoders):
        print(f"Training autoencoder {i+1}/{len(model.autoencoders)}")
        
        train_dataset = TensorDataset(latent, latent)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        optimizer = optim.Adam(ae.parameters(), lr=lr)
        criterion = nn.MSELoss()
        
        ae.train()
        for epoch in range(epochs):
            for batch_x, _ in train_loader:
                optimizer.zero_grad()
                reconstructed, new_latent = ae(batch_x)
                loss = criterion(reconstructed, batch_x)
                loss.backward()
                optimizer.step()
        
        # Получаем латентное представление для следующего автоэнкодера
        ae.eval()
        with torch.no_grad():
            _, latent = ae(latent)
    
    return model

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    
    # Получаем латентное представление из последнего автоэнкодера
    latent = X_tensor
    for ae in model.autoencoders:
        with torch.no_grad():
            _, latent = ae(latent)
            
    return latent.cpu().numpy()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    # Параметры stacked автоэнкодера
    num_layers = trial.suggest_int('num_layers', 2, 2)
    latent_dims = []
    
    # Определяем размерности слоев автоэнкодера
    dim = X_train.shape[1]
    for i in range(num_layers):
        dim = dim // 2
        dim = max(10, dim)  # Минимальный размер 10
        latent_dim = trial.suggest_int(f'latent_dim_{i}', max(8, dim//2), dim)
        latent_dims.append(latent_dim)
    
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    # Параметры градиентного бустинга
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    rocs = []
    for i in range(3):
        autoencoder = StackedAutoencoder(X_train.shape[1], latent_dims, dropout_rate)
        autoencoder = train_stacked_autoencoder(autoencoder, X_train, epochs, batch_size, lr)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params

# Восстанавливаем список размерностей слоев из best_params
latent_dims = []
for i in range(best_params['num_layers']):
    latent_dims.append(best_params[f'latent_dim_{i}'])

autoencoder = StackedAutoencoder(X_train_all.shape[1], latent_dims, best_params['dropout_rate'])
autoencoder = train_stacked_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

gb = GradientBoostingClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
gb.fit(X_train_latent, y_all_train)

y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')

[I 2025-05-11 00:15:58,672] A new study created in memory with name: no-name-038fdd6d-8d51-431b-bce6-ba284dced6b9


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:16:27,683] Trial 0 finished with value: 0.7328234031132581 and parameters: {'num_layers': 2, 'latent_dim_0': 24, 'latent_dim_1': 10, 'lr': 0.002164756621326948, 'epochs': 32, 'dropout_rate': 0.335826338092858, 'n_estimators': 118, 'max_depth': 7, 'learning_rate': 0.08088477775842576, 'subsample': 0.8818427244102958, 'min_samples_split': 2, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.7328234031132581.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:17:28,089] Trial 1 finished with value: 0.7099532244459782 and parameters: {'num_layers': 2, 'latent_dim_0': 30, 'latent_dim_1': 16, 'lr': 0.00013134112585320128, 'epochs': 36, 'dropout_rate': 0.3540723012026047, 'n_estimators': 257, 'max_depth': 8, 'learning_rate': 0.097121597505875, 'subsample': 0.9267351158341312, 'min_samples_split': 16, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.7328234031132581.


...

Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:28:35,186] Trial 18 finished with value: 0.7123495130741507 and parameters: {'num_layers': 2, 'latent_dim_0': 31, 'latent_dim_1': 12, 'lr': 0.0014776212164553575, 'epochs': 43, 'dropout_rate': 0.3811444169630075, 'n_estimators': 161, 'max_depth': 6, 'learning_rate': 0.04161735439533219, 'subsample': 0.9754009550904214, 'min_samples_split': 18, 'min_samples_leaf': 5}. Best is trial 16 with value: 0.7530864197530865.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:28:51,945] Trial 19 finished with value: 0.7242926155969635 and parameters: {'num_layers': 2, 'latent_dim_0': 22, 'latent_dim_1': 11, 'lr': 0.0004990374642630911, 'epochs': 27, 'dropout_rate': 0.3022678680009744, 'n_estimators': 54, 'max_depth': 10, 'learning_rate': 0.06811482181564152, 'subsample': 0.8321407020397452, 'min_samples_split': 14, 'min_samples_leaf': 8}. Best is trial 16 with value: 0.7530864197530865.


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded bigset: 0.7443


In [94]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        # Восстанавливаем список размерностей слоев из best_params
        latent_dims = []
        for k in range(best_params['num_layers']):
            latent_dims.append(best_params[f'latent_dim_{k}'])
            
        autoencoder = StackedAutoencoder(X_train.shape[1], latent_dims, best_params['dropout_rate'])
        autoencoder = train_stacked_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            subsample=best_params['subsample'],
            min_samples_split=best_params['min_samples_split'],
            min_samples_leaf=best_params['min_samples_leaf'],
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))

Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7290
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7263
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7300
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7337
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7057
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7490
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7433
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7472
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7391
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7392
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7483
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7355
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoen

SVC

In [95]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    # Параметры stacked автоэнкодера
    num_layers = trial.suggest_int('num_layers', 2, 2)
    latent_dims = []
    
    # Определяем размерности слоев автоэнкодера
    dim = X_train.shape[1]
    for i in range(num_layers):
        dim = dim // 2
        dim = max(10, dim)  # Минимальный размер 10
        latent_dim = trial.suggest_int(f'latent_dim_{i}', max(8, dim//2), dim)
        latent_dims.append(latent_dim)
    
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    # Параметры SVC
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    
    rocs = []
    for i in range(3):
        autoencoder = StackedAutoencoder(X_train.shape[1], latent_dims, dropout_rate)
        autoencoder = train_stacked_autoencoder(autoencoder, X_train, epochs, batch_size, lr)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        svc = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            degree=degree,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params

# Восстанавливаем список размерностей слоев из best_params
latent_dims = []
for i in range(best_params['num_layers']):
    latent_dims.append(best_params[f'latent_dim_{i}'])

autoencoder = StackedAutoencoder(X_train_all.shape[1], latent_dims, best_params['dropout_rate'])
autoencoder = train_stacked_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

svc = SVC(
    C=best_params['C'],
    kernel=best_params['kernel'],
    gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
    degree=best_params['degree'] if 'degree' in best_params else 3,
    probability=True,
    random_state=42
)
svc.fit(X_train_latent, y_all_train)

y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')

[I 2025-05-11 00:30:58,105] A new study created in memory with name: no-name-aee16232-1f01-4a2c-b4fc-4ca5a80b229b


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:31:08,200] Trial 0 finished with value: 0.6926424353960586 and parameters: {'num_layers': 2, 'latent_dim_0': 36, 'latent_dim_1': 16, 'lr': 0.0006897194401221571, 'epochs': 32, 'dropout_rate': 0.16562295825527273, 'C': 59.92422001440525, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.6926424353960586.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:31:12,600] Trial 1 finished with value: 0.7737903535004983 and parameters: {'num_layers': 2, 'latent_dim_0': 39, 'latent_dim_1': 13, 'lr': 0.0013202091268969913, 'epochs': 10, 'dropout_rate': 0.3189780436187853, 'C': 0.2798790272568221, 'kernel': 'linear'}. Best is trial 1 with value: 0.7737903535004983.


...

Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:38:31,108] Trial 18 finished with value: 0.7020167165094701 and parameters: {'num_layers': 2, 'latent_dim_0': 38, 'latent_dim_1': 12, 'lr': 0.0020282651046096755, 'epochs': 38, 'dropout_rate': 0.3490496330536294, 'C': 0.117949685447959, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 16 with value: 0.7751131048232498.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:38:38,942] Trial 19 finished with value: 0.7300053676865271 and parameters: {'num_layers': 2, 'latent_dim_0': 32, 'latent_dim_1': 14, 'lr': 0.003983097514509745, 'epochs': 26, 'dropout_rate': 0.3939545727856523, 'C': 0.26971509248381637, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 16 with value: 0.7751131048232498.


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded bigset: 0.7778


In [96]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        # Восстанавливаем список размерностей слоев из best_params
        latent_dims = []
        for k in range(best_params['num_layers']):
            latent_dims.append(best_params[f'latent_dim_{k}'])
            
        autoencoder = StackedAutoencoder(X_train.shape[1], latent_dims, best_params['dropout_rate'])
        autoencoder = train_stacked_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        svc = SVC(
            C=best_params['C'],
            kernel=best_params['kernel'],
            gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
            degree=best_params['degree'] if 'degree' in best_params else 3,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7129
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7217
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.6983
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7183
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7024
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7673
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7440
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7712
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7497
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7700
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7440
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7474
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoen

MLP

In [97]:
class SimpleAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.2):
        super(SimpleAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate)
        )
        self.decoder = nn.Sequential(
            nn.Linear(output_dim, input_dim),
            nn.BatchNorm1d(input_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate)
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent

class StackedAutoencoder(nn.Module):
    def __init__(self, input_dim, layer_dims, dropout_rate=0.2):
        super(StackedAutoencoder, self).__init__()
        self.autoencoders = nn.ModuleList()
        
        # Создаем последовательность автоэнкодеров
        prev_dim = input_dim
        for dim in layer_dims:
            self.autoencoders.append(SimpleAutoencoder(prev_dim, dim, dropout_rate))
            prev_dim = dim
        
    def forward(self, x):
        latent = x
        reconstructions = []
        latents = []
        
        # Прямой проход через все автоэнкодеры
        for ae in self.autoencoders:
            reconstructed, latent = ae(latent)
            reconstructions.append(reconstructed)
            latents.append(latent)
        
        return reconstructions, latents

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate=0.2):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)
    
def train_stacked_autoencoder(model, X_train, epochs, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    
    # Обучаем каждый автоэнкодер последовательно
    latent = X_train_tensor
    for i, ae in enumerate(model.autoencoders):
        print(f"Training autoencoder {i+1}/{len(model.autoencoders)}")
        
        train_dataset = TensorDataset(latent, latent)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        optimizer = optim.Adam(ae.parameters(), lr=lr)
        criterion = nn.MSELoss()
        
        ae.train()
        for epoch in range(epochs):
            for batch_x, _ in train_loader:
                optimizer.zero_grad()
                reconstructed, new_latent = ae(batch_x)
                loss = criterion(reconstructed, batch_x)
                loss.backward()
                optimizer.step()
        
        # Получаем латентное представление для следующего автоэнкодера
        ae.eval()
        with torch.no_grad():
            _, latent = ae(latent)
    
    return model

def train_mlp(model, X_train, y_train, epochs, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    return model

def predict_mlp(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        predictions = model(X_tensor)
    
    return predictions.cpu().numpy().flatten()

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    
    # Получаем латентное представление из последнего автоэнкодера
    latent = X_tensor
    for ae in model.autoencoders:
        with torch.no_grad():
            _, latent = ae(latent)
            
    return latent.cpu().numpy()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    # Параметры stacked автоэнкодера
    num_layers = trial.suggest_int('num_layers', 2, 2)
    latent_dims = []
    
    # Определяем размерности слоев автоэнкодера
    dim = X_train.shape[1]
    for i in range(num_layers):
        dim = dim // 2
        dim = max(10, dim)
        latent_dim = trial.suggest_int(f'latent_dim_{i}', max(8, dim//2), dim)
        latent_dims.append(latent_dim)
    
    lr_ae = trial.suggest_float('lr_ae', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs_ae = trial.suggest_int('epochs_ae', 10, 50)
    dropout_rate_ae = trial.suggest_float('dropout_rate_ae', 0.1, 0.5)
    
    # Параметры MLP
    hidden_dim_mlp = trial.suggest_int('hidden_dim_mlp', 32, 128)
    lr_mlp = trial.suggest_float('lr_mlp', 1e-4, 1e-2, log=True)
    epochs_mlp = trial.suggest_int('epochs_mlp', 10, 50)
    dropout_rate_mlp = trial.suggest_float('dropout_rate_mlp', 0.1, 0.5)
    
    rocs = []
    for i in range(3):
        autoencoder = StackedAutoencoder(X_train.shape[1], latent_dims, dropout_rate_ae)
        autoencoder = train_stacked_autoencoder(autoencoder, X_train, epochs_ae, batch_size, lr_ae)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        mlp = MLP(X_train_latent.shape[1], hidden_dim_mlp, dropout_rate_mlp)
        mlp = train_mlp(mlp, X_train_latent, y_train, epochs_mlp, batch_size, lr_mlp)
        
        y_pred_proba = predict_mlp(mlp, X_val_latent)
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params

latent_dims = []
for i in range(best_params['num_layers']):
    latent_dims.append(best_params[f'latent_dim_{i}'])

autoencoder = StackedAutoencoder(X_train_all.shape[1], latent_dims, best_params['dropout_rate_ae'])
autoencoder = train_stacked_autoencoder(autoencoder, X_train_all, best_params['epochs_ae'], 32, best_params['lr_ae'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

mlp = MLP(X_train_latent.shape[1], best_params['hidden_dim_mlp'], best_params['dropout_rate_mlp'])
mlp = train_mlp(mlp, X_train_latent, y_all_train, best_params['epochs_mlp'], 32, best_params['lr_mlp'])

y_pred_proba = predict_mlp(mlp, X_val_latent)
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 00:39:34,662] A new study created in memory with name: no-name-810d3bcf-4230-476b-996e-d81093e95ca0


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:39:48,185] Trial 0 finished with value: 0.7481213097155125 and parameters: {'num_layers': 2, 'latent_dim_0': 24, 'latent_dim_1': 19, 'lr_ae': 0.0021499959490194113, 'epochs_ae': 26, 'dropout_rate_ae': 0.23781680715923384, 'hidden_dim_mlp': 46, 'lr_mlp': 0.005827828589207834, 'epochs_mlp': 49, 'dropout_rate_mlp': 0.4904703894620587}. Best is trial 0 with value: 0.7481213097155125.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:40:00,443] Trial 1 finished with value: 0.7588758530787515 and parameters: {'num_layers': 2, 'latent_dim_0': 32, 'latent_dim_1': 19, 'lr_ae': 0.0002416464612175408, 'epochs_ae': 32, 'dropout_rate_ae': 0.21142815732056675, 'hidden_dim_mlp': 69, 'lr_mlp': 0.0010010658405874815, 'epochs_mlp': 27, 'dropout_rate_mlp': 0.43612362904253976}. Best is trial 1 with value: 0.7588758530787515.


...

Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:43:40,797] Trial 18 finished with value: 0.7604286481098076 and parameters: {'num_layers': 2, 'latent_dim_0': 30, 'latent_dim_1': 10, 'lr_ae': 0.0003615725074739476, 'epochs_ae': 31, 'dropout_rate_ae': 0.10092080370547221, 'hidden_dim_mlp': 86, 'lr_mlp': 0.00023620029265491763, 'epochs_mlp': 42, 'dropout_rate_mlp': 0.401035215668736}. Best is trial 12 with value: 0.7611187792347214.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:43:54,852] Trial 19 finished with value: 0.7593742811134115 and parameters: {'num_layers': 2, 'latent_dim_0': 29, 'latent_dim_1': 10, 'lr_ae': 0.00039410369765826143, 'epochs_ae': 28, 'dropout_rate_ae': 0.10511118266582005, 'hidden_dim_mlp': 83, 'lr_mlp': 0.0002213996990645173, 'epochs_mlp': 42, 'dropout_rate_mlp': 0.38911413250237137}. Best is trial 12 with value: 0.7611187792347214.


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded bigset: 0.7506


In [98]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        latent_dims = []
        for k in range(best_params['num_layers']):
            latent_dims.append(best_params[f'latent_dim_{k}'])
            
        autoencoder = StackedAutoencoder(X_train.shape[1], latent_dims, best_params['dropout_rate_ae'])
        autoencoder = train_stacked_autoencoder(autoencoder, X_train, best_params['epochs_ae'], 32, best_params['lr_ae'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        mlp = MLP(X_train_latent.shape[1], best_params['hidden_dim_mlp'], best_params['dropout_rate_mlp'])
        mlp = train_mlp(mlp, X_train_latent, y_train, best_params['epochs_mlp'], 32, best_params['lr_mlp'])
        
        y_pred_proba = predict_mlp(mlp, X_val_latent)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7428
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7182
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7054
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7268
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7327
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7637
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7565
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7695
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7605
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7564
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7120
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7356
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoen

Теперь блок с stacked автоэнкодером. Но у каждого из автоэнкодеров была дополнительная классифицирующая голова.

логистическая рергрессия

In [99]:
class ClassifyingSimpleAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.2):
        super(ClassifyingSimpleAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate)
        )
        self.decoder = nn.Sequential(
            nn.Linear(output_dim, input_dim),
            nn.BatchNorm1d(input_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate)
        )
        self.classifier = nn.Sequential(
            nn.Linear(output_dim, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        classification = self.classifier(latent)
        return reconstructed, latent, classification

class ClassifyingStackedAutoencoder(nn.Module):
    def __init__(self, input_dim, layer_dims, dropout_rate=0.2):
        super(ClassifyingStackedAutoencoder, self).__init__()
        self.autoencoders = nn.ModuleList()
        
        # Создаем последовательность автоэнкодеров
        prev_dim = input_dim
        for dim in layer_dims:
            self.autoencoders.append(ClassifyingSimpleAutoencoder(prev_dim, dim, dropout_rate))
            prev_dim = dim
        
    def forward(self, x):
        latent = x
        reconstructions = []
        latents = []
        classifications = []
        
        # Прямой проход через все автоэнкодеры
        for ae in self.autoencoders:
            reconstructed, latent, classification = ae(latent)
            reconstructions.append(reconstructed)
            latents.append(latent)
            classifications.append(classification)
        
        return reconstructions, latents, classifications
    
def train_classifying_stacked_autoencoder(model, X_train, y_train, epochs, batch_size, lr, classification_weights):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    latent = X_train_tensor
    for i, ae in enumerate(model.autoencoders):
        print(f"Training autoencoder {i+1}/{len(model.autoencoders)}")
        
        train_dataset = TensorDataset(latent, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        optimizer = optim.Adam(ae.parameters(), lr=lr)
        recon_criterion = nn.MSELoss()
        class_criterion = nn.BCELoss()
        
        ae.train()
        for epoch in range(epochs):
            for batch_x, batch_y in train_loader:
                optimizer.zero_grad()
                reconstructed, new_latent, classification = ae(batch_x)
                
                recon_loss = recon_criterion(reconstructed, batch_x)
                class_loss = class_criterion(classification, batch_y)
                
                recon_weight = 1.0 - classification_weights[i]
                class_weight = classification_weights[i]
                loss = recon_weight * recon_loss + class_weight * class_loss
                
                loss.backward()
                optimizer.step()
        
        # Получаем латентное представление для следующего автоэнкодера
        ae.eval()
        with torch.no_grad():
            _, latent, _ = ae(latent)
    
    return model

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    
    # Получаем латентное представление из последнего автоэнкодера
    latent = X_tensor
    for ae in model.autoencoders:
        with torch.no_grad():
            _, latent, _ = ae(latent)
            
    return latent.cpu().numpy()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    num_layers = trial.suggest_int('num_layers', 2, 2)
    latent_dims = []
    
    dim = X_train.shape[1]
    for i in range(num_layers):
        dim = dim // 2
        dim = max(10, dim)  # Минимальный размер 10
        latent_dim = trial.suggest_int(f'latent_dim_{i}', max(8, dim//2), dim)
        latent_dims.append(latent_dim)
    
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    classification_weights = []
    for i in range(num_layers):
        class_weight = trial.suggest_float(f'class_weight_{i}', 0.1, 0.9)
        classification_weights.append(class_weight)
    
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'lbfgs'])
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingStackedAutoencoder(X_train.shape[1], latent_dims, dropout_rate)
        autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, classification_weights)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        logreg = LogisticRegression(
            C=C,
            solver=solver,
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params

# Восстанавливаем список размерностей слоев из best_params
latent_dims = []
for i in range(best_params['num_layers']):
    latent_dims.append(best_params[f'latent_dim_{i}'])

# Восстанавливаем список весов классификации
classification_weights = []
for i in range(best_params['num_layers']):
    classification_weights.append(best_params[f'class_weight_{i}'])

autoencoder = ClassifyingStackedAutoencoder(X_train_all.shape[1], latent_dims, best_params['dropout_rate'])
autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, best_params['lr'], classification_weights)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

logreg = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=1000,
    random_state=42
)
logreg.fit(X_train_latent, y_all_train)

y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 00:45:26,180] A new study created in memory with name: no-name-8ed0496c-cab1-4b1c-b171-62ac391f12c7


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:45:33,969] Trial 0 finished with value: 0.7299670270684763 and parameters: {'num_layers': 2, 'latent_dim_0': 34, 'latent_dim_1': 15, 'lr': 0.0006523153058226761, 'epochs': 19, 'dropout_rate': 0.30262475655683296, 'class_weight_0': 0.5633388112542617, 'class_weight_1': 0.4531569672483491, 'C': 1.9450844155540579, 'solver': 'saga'}. Best is trial 0 with value: 0.7299670270684763.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:45:41,355] Trial 1 finished with value: 0.676079288398129 and parameters: {'num_layers': 2, 'latent_dim_0': 39, 'latent_dim_1': 17, 'lr': 0.007678859889961798, 'epochs': 24, 'dropout_rate': 0.4704783932267125, 'class_weight_0': 0.3721250046948229, 'class_weight_1': 0.7816931432912494, 'C': 0.010292474540923809, 'solver': 'saga'}. Best is trial 0 with value: 0.7299670270684763.


...

Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:47:58,249] Trial 18 finished with value: 0.7706464228203358 and parameters: {'num_layers': 2, 'latent_dim_0': 23, 'latent_dim_1': 16, 'lr': 0.00023337414004473802, 'epochs': 17, 'dropout_rate': 0.2244139924626768, 'class_weight_0': 0.7797448427878704, 'class_weight_1': 0.2481569253436561, 'C': 0.015815239854791975, 'solver': 'lbfgs'}. Best is trial 15 with value: 0.77814201364926.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:48:06,247] Trial 19 finished with value: 0.7688635840809753 and parameters: {'num_layers': 2, 'latent_dim_0': 20, 'latent_dim_1': 18, 'lr': 0.00010691061008957106, 'epochs': 25, 'dropout_rate': 0.14368414615851102, 'class_weight_0': 0.7914855618498869, 'class_weight_1': 0.18118427962074746, 'C': 0.08492999131955682, 'solver': 'lbfgs'}. Best is trial 15 with value: 0.77814201364926.


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded bigset: 0.7738


In [100]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        # Восстанавливаем список размерностей слоев из best_params
        latent_dims = []
        for k in range(best_params['num_layers']):
            latent_dims.append(best_params[f'latent_dim_{k}'])
        
        # Восстанавливаем список весов классификации
        classification_weights = []
        for k in range(best_params['num_layers']):
            classification_weights.append(best_params[f'class_weight_{k}'])
            
        autoencoder = ClassifyingStackedAutoencoder(X_train.shape[1], latent_dims, best_params['dropout_rate'])
        autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train, y_train, best_params['epochs'], 32, best_params['lr'], classification_weights)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        logreg = LogisticRegression(
            C=best_params['C'],
            solver=best_params['solver'],
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7467
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7275
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7352
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7226
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7316
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7872
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7583
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7705
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7718
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7605
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7402
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7446
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoen

Градиентный бустинг

In [101]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    num_layers = trial.suggest_int('num_layers', 2, 2)
    latent_dims = []
    
    dim = X_train.shape[1]
    for i in range(num_layers):
        dim = dim // 2
        dim = max(10, dim)
        latent_dim = trial.suggest_int(f'latent_dim_{i}', max(8, dim//2), dim)
        latent_dims.append(latent_dim)
    
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    classification_weights = []
    for i in range(num_layers):
        class_weight = trial.suggest_float(f'class_weight_{i}', 0.1, 0.9)
        classification_weights.append(class_weight)
    
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingStackedAutoencoder(X_train.shape[1], latent_dims, dropout_rate)
        autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, classification_weights)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params

latent_dims = []
for i in range(best_params['num_layers']):
    latent_dims.append(best_params[f'latent_dim_{i}'])

classification_weights = []
for i in range(best_params['num_layers']):
    classification_weights.append(best_params[f'class_weight_{i}'])

autoencoder = ClassifyingStackedAutoencoder(X_train_all.shape[1], latent_dims, best_params['dropout_rate'])
autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, best_params['lr'], classification_weights)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

gb = GradientBoostingClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
gb.fit(X_train_latent, y_all_train)

y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 00:48:46,281] A new study created in memory with name: no-name-7af06ea8-cfac-4d8e-81ca-0878835105b1


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:49:30,311] Trial 0 finished with value: 0.7336093857832989 and parameters: {'num_layers': 2, 'latent_dim_0': 34, 'latent_dim_1': 14, 'lr': 0.00011246692121613617, 'epochs': 25, 'dropout_rate': 0.26032401347053313, 'class_weight_0': 0.33317474642954437, 'class_weight_1': 0.2893251203389363, 'n_estimators': 287, 'max_depth': 4, 'learning_rate': 0.017736337519917773, 'subsample': 0.9211055515550481, 'min_samples_split': 6, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.7336093857832989.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 00:50:10,560] Trial 1 finished with value: 0.6147534698259336 and parameters: {'num_layers': 2, 'latent_dim_0': 36, 'latent_dim_1': 19, 'lr': 0.003151727987146931, 'epochs': 16, 'dropout_rate': 0.16975277869015298, 'class_weight_0': 0.8281726011706995, 'class_weight_1': 0.5317200102990268, 'n_estimators': 456, 'max_depth': 4, 'learning_rate': 0.012889774612485427, 'subsample': 0.6175008862486147, 'min_samples_split': 15, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.7336093857832989.


...

Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:02:22,821] Trial 18 finished with value: 0.6853768882754391 and parameters: {'num_layers': 2, 'latent_dim_0': 32, 'latent_dim_1': 13, 'lr': 0.0003753907055104422, 'epochs': 37, 'dropout_rate': 0.2460604844894025, 'class_weight_0': 0.7604092976265198, 'class_weight_1': 0.3833828299915191, 'n_estimators': 247, 'max_depth': 7, 'learning_rate': 0.05856850692943994, 'subsample': 0.828049535793467, 'min_samples_split': 11, 'min_samples_leaf': 2}. Best is trial 13 with value: 0.7483705237328425.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:03:08,793] Trial 19 finished with value: 0.7365999539912583 and parameters: {'num_layers': 2, 'latent_dim_0': 23, 'latent_dim_1': 17, 'lr': 0.00014082937965349574, 'epochs': 21, 'dropout_rate': 0.1506648576412569, 'class_weight_0': 0.11004460195159863, 'class_weight_1': 0.4699372387276056, 'n_estimators': 344, 'max_depth': 6, 'learning_rate': 0.010089644767557615, 'subsample': 0.6537860902148742, 'min_samples_split': 4, 'min_samples_leaf': 5}. Best is trial 13 with value: 0.7483705237328425.


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded bigset: 0.7400


In [102]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        latent_dims = []
        for k in range(best_params['num_layers']):
            latent_dims.append(best_params[f'latent_dim_{k}'])
        
        classification_weights = []
        for k in range(best_params['num_layers']):
            classification_weights.append(best_params[f'class_weight_{k}'])
            
        autoencoder = ClassifyingStackedAutoencoder(X_train.shape[1], latent_dims, best_params['dropout_rate'])
        autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train, y_train, best_params['epochs'], 32, best_params['lr'], classification_weights)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            subsample=best_params['subsample'],
            min_samples_split=best_params['min_samples_split'],
            min_samples_leaf=best_params['min_samples_leaf'],
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7190
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.6730
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7046
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7251
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7076
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7366
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7360
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7284
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7192
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7111
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7390
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7419
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoen

SVC

In [103]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    num_layers = trial.suggest_int('num_layers', 2, 2)
    latent_dims = []
    
    dim = X_train.shape[1]
    for i in range(num_layers):
        dim = dim // 2
        dim = max(10, dim)
        latent_dim = trial.suggest_int(f'latent_dim_{i}', max(8, dim//2), dim)
        latent_dims.append(latent_dim)
    
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    classification_weights = []
    for i in range(num_layers):
        class_weight = trial.suggest_float(f'class_weight_{i}', 0.1, 0.9)
        classification_weights.append(class_weight)
    
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingStackedAutoencoder(X_train.shape[1], latent_dims, dropout_rate)
        autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, classification_weights)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        svc = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            degree=degree,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params

latent_dims = []
for i in range(best_params['num_layers']):
    latent_dims.append(best_params[f'latent_dim_{i}'])

classification_weights = []
for i in range(best_params['num_layers']):
    classification_weights.append(best_params[f'class_weight_{i}'])

autoencoder = ClassifyingStackedAutoencoder(X_train_all.shape[1], latent_dims, best_params['dropout_rate'])
autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, best_params['lr'], classification_weights)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

svc = SVC(
    C=best_params['C'],
    kernel=best_params['kernel'],
    gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
    degree=best_params['degree'] if 'degree' in best_params else 3,
    probability=True,
    random_state=42
)
svc.fit(X_train_latent, y_all_train)

y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 01:08:43,937] A new study created in memory with name: no-name-e7cbfb01-98f6-4936-9cf5-11271dd7435e


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:08:48,917] Trial 0 finished with value: 0.7304846254121617 and parameters: {'num_layers': 2, 'latent_dim_0': 24, 'latent_dim_1': 11, 'lr': 0.003942319289944774, 'epochs': 12, 'dropout_rate': 0.4769714232170974, 'class_weight_0': 0.48618961483629486, 'class_weight_1': 0.31226014152578896, 'C': 0.3744913014182304, 'kernel': 'linear'}. Best is trial 0 with value: 0.7304846254121617.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:09:02,876] Trial 1 finished with value: 0.7551088873552642 and parameters: {'num_layers': 2, 'latent_dim_0': 24, 'latent_dim_1': 13, 'lr': 0.0012411891643998997, 'epochs': 36, 'dropout_rate': 0.12238603336386761, 'class_weight_0': 0.10254174079483924, 'class_weight_1': 0.18666188627964297, 'C': 0.6790191707763609, 'kernel': 'linear'}. Best is trial 1 with value: 0.7551088873552642.


...

Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:16:58,927] Trial 18 finished with value: 0.7666973391611073 and parameters: {'num_layers': 2, 'latent_dim_0': 22, 'latent_dim_1': 15, 'lr': 0.00026479449525636863, 'epochs': 32, 'dropout_rate': 0.24327533018641403, 'class_weight_0': 0.3415375449915874, 'class_weight_1': 0.385662528239557, 'C': 0.10691869848195294, 'kernel': 'linear'}. Best is trial 12 with value: 0.7719883444521126.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:17:06,337] Trial 19 finished with value: 0.6753124760371136 and parameters: {'num_layers': 2, 'latent_dim_0': 22, 'latent_dim_1': 13, 'lr': 0.0007261349170882724, 'epochs': 22, 'dropout_rate': 0.1693462310619583, 'class_weight_0': 0.41693697300322574, 'class_weight_1': 0.6633711016936925, 'C': 5.534526343596685, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 12 with value: 0.7719883444521126.


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded bigset: 0.7749


In [104]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        latent_dims = []
        for k in range(best_params['num_layers']):
            latent_dims.append(best_params[f'latent_dim_{k}'])
        
        classification_weights = []
        for k in range(best_params['num_layers']):
            classification_weights.append(best_params[f'class_weight_{k}'])
            
        autoencoder = ClassifyingStackedAutoencoder(X_train.shape[1], latent_dims, best_params['dropout_rate'])
        autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train, y_train, best_params['epochs'], 32, best_params['lr'], classification_weights)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        svc = SVC(
            C=best_params['C'],
            kernel=best_params['kernel'],
            gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
            degree=best_params['degree'] if 'degree' in best_params else 3,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7053
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7007
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7349
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7087
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7216
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7258
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7571
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7536
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7672
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7493
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7330
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7190
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoen

MLP

In [105]:
class CombinedClassifier(nn.Module):
    def __init__(self, stacked_autoencoder, pgs_input_dim, hidden_dim=64, dropout_rate=0.2):
        super(CombinedClassifier, self).__init__()
        self.stacked_autoencoder = stacked_autoencoder
        self.pgs_branch = nn.Sequential(
            nn.Linear(pgs_input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2)
        )
        self.combined_classifier = nn.Sequential(
            nn.Linear(hidden_dim // 2 + stacked_autoencoder.autoencoders[-1].encoder[0].out_features, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x_snp, x_pgs):
        latent = x_snp
        for ae in self.stacked_autoencoder.autoencoders:
            _, latent, _ = ae(latent)
        
        pgs_features = self.pgs_branch(x_pgs)
        combined = torch.cat([latent, pgs_features], dim=1)
        output = self.combined_classifier(combined)
        return output

def train_combined_classifier(model, X_train_snp, X_train_pgs, y_train, epochs_freeze, epochs_unfreeze, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    X_train_snp_tensor = torch.FloatTensor(X_train_snp).to(device)
    X_train_pgs_tensor = torch.FloatTensor(X_train_pgs).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_snp_tensor, X_train_pgs_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    criterion = nn.BCELoss()
    
    for param in model.stacked_autoencoder.parameters():
        param.requires_grad = False
    
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    model.train()
    for epoch in range(epochs_freeze):
        for batch_snp, batch_pgs, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_snp, batch_pgs)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    for param in model.stacked_autoencoder.parameters():
        param.requires_grad = True
    
    optimizer = optim.Adam(model.parameters(), lr=lr/10)
    
    for epoch in range(epochs_unfreeze):
        for batch_snp, batch_pgs, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_snp, batch_pgs)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    return model

def predict_combined(model, X_snp, X_pgs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    X_snp_tensor = torch.FloatTensor(X_snp).to(device)
    X_pgs_tensor = torch.FloatTensor(X_pgs).to(device)
    
    with torch.no_grad():
        predictions = model(X_snp_tensor, X_pgs_tensor)
    
    return predictions.cpu().numpy().flatten()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    num_layers = trial.suggest_int('num_layers', 2, 2)
    latent_dims = []
    
    dim = X_train.shape[1]
    for i in range(num_layers):
        dim = dim // 2
        dim = max(10, dim)
        latent_dim = trial.suggest_int(f'latent_dim_{i}', max(8, dim//2), dim)
        latent_dims.append(latent_dim)
    
    lr_ae = trial.suggest_float('lr_ae', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs_ae = trial.suggest_int('epochs_ae', 10, 50)
    dropout_rate_ae = trial.suggest_float('dropout_rate_ae', 0.1, 0.5)
    
    classification_weights = []
    for i in range(num_layers):
        class_weight = trial.suggest_float(f'class_weight_{i}', 0.1, 0.9)
        classification_weights.append(class_weight)
    
    hidden_dim_combined = trial.suggest_int('hidden_dim_combined', 32, 128)
    lr_combined = trial.suggest_float('lr_combined', 1e-4, 1e-2, log=True)
    epochs_freeze = trial.suggest_int('epochs_freeze', 5, 20)
    epochs_unfreeze = trial.suggest_int('epochs_unfreeze', 5, 20)
    dropout_rate_combined = trial.suggest_float('dropout_rate_combined', 0.1, 0.5)
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingStackedAutoencoder(X_train.shape[1], latent_dims, dropout_rate_ae)
        autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train, y_train, epochs_ae, batch_size, lr_ae, classification_weights)
        
        combined_model = CombinedClassifier(autoencoder, X_pgs_train.shape[1], hidden_dim_combined, dropout_rate_combined)
        combined_model = train_combined_classifier(combined_model, X_train, X_pgs_train, y_train, epochs_freeze, epochs_unfreeze, batch_size, lr_combined)
        
        y_pred_proba = predict_combined(combined_model, X_val, X_pgs_val)
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params

latent_dims = []
for i in range(best_params['num_layers']):
    latent_dims.append(best_params[f'latent_dim_{i}'])

classification_weights = []
for i in range(best_params['num_layers']):
    classification_weights.append(best_params[f'class_weight_{i}'])

autoencoder = ClassifyingStackedAutoencoder(X_train_all.shape[1], latent_dims, best_params['dropout_rate_ae'])
autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train_all, y_all_train, best_params['epochs_ae'], 32, best_params['lr_ae'], classification_weights)

combined_model = CombinedClassifier(autoencoder, X_train_pgs.shape[1], best_params['hidden_dim_combined'], best_params['dropout_rate_combined'])
combined_model = train_combined_classifier(
    combined_model, X_train_all, X_train_pgs, y_all_train, 
    best_params['epochs_freeze'], best_params['epochs_unfreeze'], 32, best_params['lr_combined']
)

y_pred_proba = predict_combined(combined_model, X_val_all, X_val_pgs)
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 01:19:37,871] A new study created in memory with name: no-name-4bc03b76-221c-4b48-a321-1cfea0df1778


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:19:52,398] Trial 0 finished with value: 0.7548309178743962 and parameters: {'num_layers': 2, 'latent_dim_0': 22, 'latent_dim_1': 20, 'lr_ae': 0.0002895712921479693, 'epochs_ae': 36, 'dropout_rate_ae': 0.14616552266634267, 'class_weight_0': 0.3121484661096837, 'class_weight_1': 0.789946859094628, 'hidden_dim_combined': 128, 'lr_combined': 0.00030239100531223, 'epochs_freeze': 8, 'epochs_unfreeze': 7, 'dropout_rate_combined': 0.2529728456312813}. Best is trial 0 with value: 0.7548309178743962.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:20:02,295] Trial 1 finished with value: 0.7491565064028833 and parameters: {'num_layers': 2, 'latent_dim_0': 23, 'latent_dim_1': 14, 'lr_ae': 0.00012708116071309734, 'epochs_ae': 15, 'dropout_rate_ae': 0.12967136815027627, 'class_weight_0': 0.4706799739555979, 'class_weight_1': 0.5239939210464795, 'hidden_dim_combined': 41, 'lr_combined': 0.004274033537436669, 'epochs_freeze': 6, 'epochs_unfreeze': 15, 'dropout_rate_combined': 0.47066506610210235}. Best is trial 0 with value: 0.7548309178743962.


...

Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:25:12,318] Trial 18 finished with value: 0.7426002607162028 and parameters: {'num_layers': 2, 'latent_dim_0': 24, 'latent_dim_1': 15, 'lr_ae': 0.0012940547373170557, 'epochs_ae': 36, 'dropout_rate_ae': 0.40717819578098835, 'class_weight_0': 0.2134063583920529, 'class_weight_1': 0.6653445974382471, 'hidden_dim_combined': 32, 'lr_combined': 0.0037753082847481133, 'epochs_freeze': 11, 'epochs_unfreeze': 20, 'dropout_rate_combined': 0.4495725156588932}. Best is trial 13 with value: 0.7642627099148838.


Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2
Training autoencoder 1/2
Training autoencoder 2/2


[I 2025-05-11 01:25:36,780] Trial 19 finished with value: 0.7272256728778467 and parameters: {'num_layers': 2, 'latent_dim_0': 28, 'latent_dim_1': 18, 'lr_ae': 0.0005632981268866106, 'epochs_ae': 45, 'dropout_rate_ae': 0.22530186209830122, 'class_weight_0': 0.3841861300017362, 'class_weight_1': 0.24449377297465869, 'hidden_dim_combined': 70, 'lr_combined': 0.001754463750821921, 'epochs_freeze': 19, 'epochs_unfreeze': 18, 'dropout_rate_combined': 0.35157854982256975}. Best is trial 13 with value: 0.7642627099148838.


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded bigset: 0.7548


In [106]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        latent_dims = []
        for k in range(best_params['num_layers']):
            latent_dims.append(best_params[f'latent_dim_{k}'])
        
        classification_weights = []
        for k in range(best_params['num_layers']):
            classification_weights.append(best_params[f'class_weight_{k}'])
            
        autoencoder = ClassifyingStackedAutoencoder(X_train.shape[1], latent_dims, best_params['dropout_rate_ae'])
        autoencoder = train_classifying_stacked_autoencoder(autoencoder, X_train, y_train, best_params['epochs_ae'], 32, best_params['lr_ae'], classification_weights)
        
        combined_model = CombinedClassifier(autoencoder, train_pgs.shape[1], best_params['hidden_dim_combined'], best_params['dropout_rate_combined'])
        combined_model = train_combined_classifier(
            combined_model, X_train, train_pgs, y_train, 
            best_params['epochs_freeze'], best_params['epochs_unfreeze'], 32, best_params['lr_combined']
        )
        
        y_pred_proba = predict_combined(combined_model, X_val, test_pgs)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7010
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.6828
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7120
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.6922
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.6917
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7689
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7620
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7458
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7525
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7438
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7103
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoencoded: 0.7051
Training autoencoder 1/2
Training autoencoder 2/2
ROC-AUC autoen

Дальше идет блок с denoising автоэнкодером

логистическая регрессия

In [107]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim=128, dropout_rate=0.2):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.LeakyReLU(0.2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, input_dim)
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent
    
def train_denoising_autoencoder(model, X_train, epochs, batch_size, lr, noise_level=0.1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, batch_target in train_loader:
            # Add noise to input data
            noisy_batch = batch_x + noise_level * torch.randn_like(batch_x)
            
            optimizer.zero_grad()
            reconstructed, _ = model(noisy_batch)
            # Target is the original data (not noisy)
            loss = criterion(reconstructed, batch_target)
            loss.backward()
            optimizer.step()
    
    return model

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        _, latent = model(X_tensor)
    return latent.cpu().numpy()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    noise_level = trial.suggest_float('noise_level', 0.05, 0.3)
    
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'lbfgs'])
    
    rocs = []
    for i in range(3):
        autoencoder = DenoisingAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_denoising_autoencoder(autoencoder, X_train, epochs, batch_size, lr, noise_level)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        logreg = LogisticRegression(
            C=C,
            solver=solver,
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = DenoisingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_denoising_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'], best_params['noise_level'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

logreg = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=1000,
    random_state=42
)
logreg.fit(X_train_latent, y_all_train)

y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 01:28:11,144] A new study created in memory with name: no-name-bc931d90-146e-4c64-be81-133ca9d49633
[I 2025-05-11 01:28:24,018] Trial 0 finished with value: 0.755923625488843 and parameters: {'latent_dim': 53, 'hidden_dim': 85, 'lr': 0.004380643411001898, 'epochs': 45, 'dropout_rate': 0.16260611864571684, 'noise_level': 0.27437748050642763, 'C': 0.39478393315988086, 'solver': 'saga'}. Best is trial 0 with value: 0.755923625488843.
[I 2025-05-11 01:28:35,432] Trial 1 finished with value: 0.774978912660072 and parameters: {'latent_dim': 36, 'hidden_dim': 100, 'lr': 0.0013484225419722706, 'epochs': 43, 'dropout_rate': 0.2931141660243142, 'noise_level': 0.16623064656686903, 'C': 0.03974222378246643, 'solver': 'liblinear'}. Best is trial 1 with value: 0.774978912660072.
[I 2025-05-11 01:28:48,054] Trial 2 finished with value: 0.7518020090483858 and parameters: {'latent_dim': 74, 'hidden_dim': 238, 'lr': 0.0008057519346256342, 'epochs': 39, 'dropout_rate': 0.30498190183495355, 

ROC-AUC autoencoded bigset: 0.7796


In [108]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = DenoisingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_denoising_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'], best_params['noise_level'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        logreg = LogisticRegression(
            C=best_params['C'],
            solver=best_params['solver'],
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7585
ROC-AUC autoencoded: 0.7465
ROC-AUC autoencoded: 0.7550
ROC-AUC autoencoded: 0.7459
ROC-AUC autoencoded: 0.7570
ROC-AUC autoencoded: 0.7683
ROC-AUC autoencoded: 0.7795
ROC-AUC autoencoded: 0.7716
ROC-AUC autoencoded: 0.7657
ROC-AUC autoencoded: 0.7808
ROC-AUC autoencoded: 0.7456
ROC-AUC autoencoded: 0.7379
ROC-AUC autoencoded: 0.7492
ROC-AUC autoencoded: 0.7397
ROC-AUC autoencoded: 0.7366
ROC-AUC autoencoded: 0.6979
ROC-AUC autoencoded: 0.7029
ROC-AUC autoencoded: 0.7052
ROC-AUC autoencoded: 0.7059
ROC-AUC autoencoded: 0.7026
ROC-AUC autoencoded: 0.7606
ROC-AUC autoencoded: 0.7662
ROC-AUC autoencoded: 0.7575
ROC-AUC autoencoded: 0.7540
ROC-AUC autoencoded: 0.7617
среднее 0.7460874333307989


Градиентный бустинг

In [109]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    noise_level = trial.suggest_float('noise_level', 0.05, 0.3)
    
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    rocs = []
    for i in range(3):
        autoencoder = DenoisingAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_denoising_autoencoder(autoencoder, X_train, epochs, batch_size, lr, noise_level)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = DenoisingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_denoising_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'], best_params['noise_level'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

gb = GradientBoostingClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
gb.fit(X_train_latent, y_all_train)

y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 01:33:05,489] A new study created in memory with name: no-name-9d15a218-343d-4360-9aff-606d6f19075c
[I 2025-05-11 01:33:23,141] Trial 0 finished with value: 0.7319607392071159 and parameters: {'latent_dim': 61, 'hidden_dim': 218, 'lr': 0.0010273136071119065, 'epochs': 22, 'dropout_rate': 0.46996071840092857, 'noise_level': 0.09998413935755333, 'n_estimators': 50, 'max_depth': 5, 'learning_rate': 0.017636274070675844, 'subsample': 0.9199193685037758, 'min_samples_split': 10, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.7319607392071159.
[I 2025-05-11 01:34:51,313] Trial 1 finished with value: 0.7217621348056132 and parameters: {'latent_dim': 31, 'hidden_dim': 179, 'lr': 0.0005255648473278012, 'epochs': 15, 'dropout_rate': 0.33065627306810846, 'noise_level': 0.23836730798653627, 'n_estimators': 482, 'max_depth': 8, 'learning_rate': 0.038385832100862896, 'subsample': 0.6530459092309695, 'min_samples_split': 3, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7

ROC-AUC autoencoded bigset: 0.7471


In [110]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = DenoisingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_denoising_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'], best_params['noise_level'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            subsample=best_params['subsample'],
            min_samples_split=best_params['min_samples_split'],
            min_samples_leaf=best_params['min_samples_leaf'],
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7401
ROC-AUC autoencoded: 0.7344
ROC-AUC autoencoded: 0.7396
ROC-AUC autoencoded: 0.7348
ROC-AUC autoencoded: 0.7268
ROC-AUC autoencoded: 0.7286
ROC-AUC autoencoded: 0.7372
ROC-AUC autoencoded: 0.7391
ROC-AUC autoencoded: 0.7406
ROC-AUC autoencoded: 0.7234
ROC-AUC autoencoded: 0.7301
ROC-AUC autoencoded: 0.7309
ROC-AUC autoencoded: 0.7453
ROC-AUC autoencoded: 0.7327
ROC-AUC autoencoded: 0.7307
ROC-AUC autoencoded: 0.7394
ROC-AUC autoencoded: 0.7343
ROC-AUC autoencoded: 0.7272
ROC-AUC autoencoded: 0.7097
ROC-AUC autoencoded: 0.7419
ROC-AUC autoencoded: 0.7550
ROC-AUC autoencoded: 0.7334
ROC-AUC autoencoded: 0.7423
ROC-AUC autoencoded: 0.7800
ROC-AUC autoencoded: 0.7580
среднее 0.7374219450250097


SVC

In [111]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    noise_level = trial.suggest_float('noise_level', 0.05, 0.3)
    
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    
    rocs = []
    for i in range(3):
        autoencoder = DenoisingAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_denoising_autoencoder(autoencoder, X_train, epochs, batch_size, lr, noise_level)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        svc = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            degree=degree,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = DenoisingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_denoising_autoencoder(autoencoder, X_train_all, best_params['epochs'], 32, best_params['lr'], best_params['noise_level'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

svc = SVC(
    C=best_params['C'],
    kernel=best_params['kernel'],
    gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
    degree=best_params['degree'] if 'degree' in best_params else 3,
    probability=True,
    random_state=42
)
svc.fit(X_train_latent, y_all_train)

y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 01:51:53,751] A new study created in memory with name: no-name-e1705c5e-37db-4fa0-ac7d-5ba47062114e
[I 2025-05-11 01:51:59,596] Trial 0 finished with value: 0.6706828464074842 and parameters: {'latent_dim': 88, 'hidden_dim': 156, 'lr': 0.006671444345683114, 'epochs': 18, 'dropout_rate': 0.12009061121472664, 'noise_level': 0.27502223925424213, 'C': 1.218781456763991, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.6706828464074842.
[I 2025-05-11 01:52:08,941] Trial 1 finished with value: 0.7573326432022084 and parameters: {'latent_dim': 58, 'hidden_dim': 169, 'lr': 0.006584716060344973, 'epochs': 28, 'dropout_rate': 0.4825236613159103, 'noise_level': 0.28291183995202773, 'C': 0.23646249452041543, 'kernel': 'linear'}. Best is trial 1 with value: 0.7573326432022084.
[I 2025-05-11 01:52:19,578] Trial 2 finished with value: 0.6113411548194158 and parameters: {'latent_dim': 95, 'hidden_dim': 130, 'lr': 0.009191694725775343, 'epochs': 34, 'dropout_rate': 0.2

ROC-AUC autoencoded bigset: 0.7468


In [112]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = DenoisingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_denoising_autoencoder(autoencoder, X_train, best_params['epochs'], 32, best_params['lr'], best_params['noise_level'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        svc = SVC(
            C=best_params['C'],
            kernel=best_params['kernel'],
            gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
            degree=best_params['degree'] if 'degree' in best_params else 3,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7185
ROC-AUC autoencoded: 0.7192
ROC-AUC autoencoded: 0.7252
ROC-AUC autoencoded: 0.7201
ROC-AUC autoencoded: 0.6979
ROC-AUC autoencoded: 0.7585
ROC-AUC autoencoded: 0.7507
ROC-AUC autoencoded: 0.7579
ROC-AUC autoencoded: 0.7398
ROC-AUC autoencoded: 0.7648
ROC-AUC autoencoded: 0.7339
ROC-AUC autoencoded: 0.7331
ROC-AUC autoencoded: 0.7108
ROC-AUC autoencoded: 0.7147
ROC-AUC autoencoded: 0.7144
ROC-AUC autoencoded: 0.6540
ROC-AUC autoencoded: 0.6254
ROC-AUC autoencoded: 0.6537
ROC-AUC autoencoded: 0.6367
ROC-AUC autoencoded: 0.6598
ROC-AUC autoencoded: 0.7173
ROC-AUC autoencoded: 0.7379
ROC-AUC autoencoded: 0.7310
ROC-AUC autoencoded: 0.7227
ROC-AUC autoencoded: 0.7217
среднее 0.7127905422628076


MLP

In [113]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate=0.2):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

def train_mlp(model, X_train, y_train, epochs, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    return model

def predict_mlp(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        predictions = model(X_tensor)
    
    return predictions.cpu().numpy().flatten()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim_ae = trial.suggest_int('hidden_dim_ae', 64, 256)
    lr_ae = trial.suggest_float('lr_ae', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs_ae = trial.suggest_int('epochs_ae', 10, 50)
    dropout_rate_ae = trial.suggest_float('dropout_rate_ae', 0.1, 0.5)
    noise_level = trial.suggest_float('noise_level', 0.05, 0.3)
    
    hidden_dim_mlp = trial.suggest_int('hidden_dim_mlp', 32, 128)
    lr_mlp = trial.suggest_float('lr_mlp', 1e-4, 1e-2, log=True)
    epochs_mlp = trial.suggest_int('epochs_mlp', 10, 50)
    dropout_rate_mlp = trial.suggest_float('dropout_rate_mlp', 0.1, 0.5)
    
    rocs = []
    for i in range(3):
        autoencoder = DenoisingAutoencoder(X_train.shape[1], latent_dim, hidden_dim_ae, dropout_rate_ae)
        autoencoder = train_denoising_autoencoder(autoencoder, X_train, epochs_ae, batch_size, lr_ae, noise_level)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        mlp = MLP(X_train_latent.shape[1], hidden_dim_mlp, dropout_rate_mlp)
        mlp = train_mlp(mlp, X_train_latent, y_train, epochs_mlp, batch_size, lr_mlp)
        
        y_pred_proba = predict_mlp(mlp, X_val_latent)
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = DenoisingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
autoencoder = train_denoising_autoencoder(autoencoder, X_train_all, best_params['epochs_ae'], 32, best_params['lr_ae'], best_params['noise_level'])

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

mlp = MLP(X_train_latent.shape[1], best_params['hidden_dim_mlp'], best_params['dropout_rate_mlp'])
mlp = train_mlp(mlp, X_train_latent, y_all_train, best_params['epochs_mlp'], 32, best_params['lr_mlp'])

y_pred_proba = predict_mlp(mlp, X_val_latent)
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 03:04:20,608] A new study created in memory with name: no-name-d29bbdf6-d1fc-41f8-a8e1-9dd945a21f93
[I 2025-05-11 03:04:37,761] Trial 0 finished with value: 0.7012690744574802 and parameters: {'latent_dim': 54, 'hidden_dim_ae': 199, 'lr_ae': 0.00015102793203035353, 'epochs_ae': 39, 'dropout_rate_ae': 0.402164643059144, 'noise_level': 0.2765553742368755, 'hidden_dim_mlp': 125, 'lr_mlp': 0.008243217728860911, 'epochs_mlp': 23, 'dropout_rate_mlp': 0.2297201414242827}. Best is trial 0 with value: 0.7012690744574802.
[I 2025-05-11 03:04:53,298] Trial 1 finished with value: 0.7130204738900391 and parameters: {'latent_dim': 39, 'hidden_dim_ae': 91, 'lr_ae': 0.0001390451840505409, 'epochs_ae': 41, 'dropout_rate_ae': 0.3147414837925032, 'noise_level': 0.2600711961795529, 'hidden_dim_mlp': 86, 'lr_mlp': 0.0006139518631047808, 'epochs_mlp': 25, 'dropout_rate_mlp': 0.20035154055958038}. Best is trial 1 with value: 0.7130204738900391.
[I 2025-05-11 03:05:03,795] Trial 2 finished with 

ROC-AUC autoencoded bigset: 0.7461


In [114]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = DenoisingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
        autoencoder = train_denoising_autoencoder(autoencoder, X_train, best_params['epochs_ae'], 32, best_params['lr_ae'], best_params['noise_level'])
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        mlp = MLP(X_train_latent.shape[1], best_params['hidden_dim_mlp'], best_params['dropout_rate_mlp'])
        mlp = train_mlp(mlp, X_train_latent, y_train, best_params['epochs_mlp'], 32, best_params['lr_mlp'])
        
        y_pred_proba = predict_mlp(mlp, X_val_latent)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7545
ROC-AUC autoencoded: 0.7432
ROC-AUC autoencoded: 0.7457
ROC-AUC autoencoded: 0.7407
ROC-AUC autoencoded: 0.7407
ROC-AUC autoencoded: 0.7522
ROC-AUC autoencoded: 0.7611
ROC-AUC autoencoded: 0.7760
ROC-AUC autoencoded: 0.7702
ROC-AUC autoencoded: 0.7726
ROC-AUC autoencoded: 0.7418
ROC-AUC autoencoded: 0.7116
ROC-AUC autoencoded: 0.7239
ROC-AUC autoencoded: 0.7377
ROC-AUC autoencoded: 0.7374
ROC-AUC autoencoded: 0.7050
ROC-AUC autoencoded: 0.6960
ROC-AUC autoencoded: 0.6974
ROC-AUC autoencoded: 0.7053
ROC-AUC autoencoded: 0.6835
ROC-AUC autoencoded: 0.7534
ROC-AUC autoencoded: 0.7357
ROC-AUC autoencoded: 0.7471
ROC-AUC autoencoded: 0.7759
ROC-AUC autoencoded: 0.7509
среднее 0.7383818946090119


Теперь блок с denoising автоэнкодером с дополнительной классифицирующей головой

Логистическая регрессия

In [115]:
class ClassifyingDenoisingAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim=128, dropout_rate=0.2):
        super(ClassifyingDenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.LeakyReLU(0.2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, input_dim)
        )
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        classification = self.classifier(latent)
        return reconstructed, latent, classification
    
def train_classifying_denoising_autoencoder(model, X_train, y_train, epochs, batch_size, lr, noise_level=0.1, classification_weight=0.5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    recon_criterion = nn.MSELoss()
    class_criterion = nn.BCELoss()
    
    model.train()
    for epoch in range(epochs):
        for batch_x, batch_target, batch_y in train_loader:
            # Add noise to input data
            noisy_batch = batch_x + noise_level * torch.randn_like(batch_x)
            
            optimizer.zero_grad()
            reconstructed, _, classification = model(noisy_batch)
            
            # Reconstruction loss (target is the original data)
            recon_loss = recon_criterion(reconstructed, batch_target)
            
            # Classification loss
            class_loss = class_criterion(classification, batch_y)
            
            # Combined loss with weights
            loss = (1 - classification_weight) * recon_loss + classification_weight * class_loss
            
            loss.backward()
            optimizer.step()
    
    return model

def get_latent_features(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        _, latent, _ = model(X_tensor)
    return latent.cpu().numpy()

def get_classification(model, X):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        _, _, classification = model(X_tensor)
    return classification.cpu().numpy().flatten()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    noise_level = trial.suggest_float('noise_level', 0.05, 0.3)
    classification_weight = trial.suggest_float('classification_weight', 0.1, 0.9)
    
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'lbfgs'])
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingDenoisingAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_classifying_denoising_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, noise_level, classification_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        logreg = LogisticRegression(
            C=C,
            solver=solver,
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingDenoisingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_classifying_denoising_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, 
    best_params['lr'], best_params['noise_level'], best_params['classification_weight']
)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

logreg = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=1000,
    random_state=42
)
logreg.fit(X_train_latent, y_all_train)

y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 03:11:03,002] A new study created in memory with name: no-name-eea38762-2b56-4012-889f-5b9072e4ff32
[I 2025-05-11 03:11:19,503] Trial 0 finished with value: 0.5925350816655165 and parameters: {'latent_dim': 79, 'hidden_dim': 194, 'lr': 0.0005281179250256072, 'epochs': 29, 'dropout_rate': 0.32932165622191506, 'noise_level': 0.23986132087917084, 'classification_weight': 0.7454312435149204, 'C': 0.29526264956716886, 'solver': 'liblinear'}. Best is trial 0 with value: 0.5925350816655165.
[I 2025-05-11 03:11:27,734] Trial 1 finished with value: 0.61833831761368 and parameters: {'latent_dim': 18, 'hidden_dim': 114, 'lr': 0.0056313477951829925, 'epochs': 16, 'dropout_rate': 0.344805016149899, 'noise_level': 0.14868804705366334, 'classification_weight': 0.7417313345606125, 'C': 10.294461681365386, 'solver': 'liblinear'}. Best is trial 1 with value: 0.61833831761368.
[I 2025-05-11 03:11:44,295] Trial 2 finished with value: 0.6919331339621196 and parameters: {'latent_dim': 19, 'hid

ROC-AUC autoencoded bigset: 0.7592


In [116]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingDenoisingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_classifying_denoising_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs'], 32, 
            best_params['lr'], best_params['noise_level'], best_params['classification_weight']
        )
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        logreg = LogisticRegression(
            C=best_params['C'],
            solver=best_params['solver'],
            max_iter=1000,
            random_state=42
        )
        logreg.fit(X_train_latent, y_train)
        
        y_pred_proba = logreg.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.6874
ROC-AUC autoencoded: 0.7028
ROC-AUC autoencoded: 0.7000
ROC-AUC autoencoded: 0.7154
ROC-AUC autoencoded: 0.7055
ROC-AUC autoencoded: 0.7609
ROC-AUC autoencoded: 0.7443
ROC-AUC autoencoded: 0.7383
ROC-AUC autoencoded: 0.7263
ROC-AUC autoencoded: 0.7650
ROC-AUC autoencoded: 0.7052
ROC-AUC autoencoded: 0.7266
ROC-AUC autoencoded: 0.7405
ROC-AUC autoencoded: 0.7290
ROC-AUC autoencoded: 0.7388
ROC-AUC autoencoded: 0.6758
ROC-AUC autoencoded: 0.6570
ROC-AUC autoencoded: 0.6824
ROC-AUC autoencoded: 0.6461
ROC-AUC autoencoded: 0.6762
ROC-AUC autoencoded: 0.7370
ROC-AUC autoencoded: 0.7341
ROC-AUC autoencoded: 0.7173
ROC-AUC autoencoded: 0.7103
ROC-AUC autoencoded: 0.7462
среднее 0.7147295091771378


градиентный бустинг

In [117]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    noise_level = trial.suggest_float('noise_level', 0.05, 0.3)
    classification_weight = trial.suggest_float('classification_weight', 0.1, 0.9)
    
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingDenoisingAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_classifying_denoising_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, noise_level, classification_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingDenoisingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_classifying_denoising_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, 
    best_params['lr'], best_params['noise_level'], best_params['classification_weight']
)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

gb = GradientBoostingClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
gb.fit(X_train_latent, y_all_train)

y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 03:16:58,306] A new study created in memory with name: no-name-42d445b3-04ea-49ca-9581-afd642fafc0d
[I 2025-05-11 03:18:17,400] Trial 0 finished with value: 0.7096656698105974 and parameters: {'latent_dim': 66, 'hidden_dim': 115, 'lr': 0.0005702531753660865, 'epochs': 10, 'dropout_rate': 0.2845153503795229, 'noise_level': 0.1855684434640053, 'classification_weight': 0.2983997300846134, 'n_estimators': 320, 'max_depth': 9, 'learning_rate': 0.05905341431051565, 'subsample': 0.6490809424048851, 'min_samples_split': 20, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.7096656698105974.
[I 2025-05-11 03:18:57,994] Trial 1 finished with value: 0.592362548884288 and parameters: {'latent_dim': 74, 'hidden_dim': 245, 'lr': 0.0008873764003811404, 'epochs': 44, 'dropout_rate': 0.3517091198621064, 'noise_level': 0.12022404482000147, 'classification_weight': 0.224784818898676, 'n_estimators': 171, 'max_depth': 3, 'learning_rate': 0.011606958151429496, 'subsample': 0.9249098109036

ROC-AUC autoencoded bigset: 0.7287


In [118]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingDenoisingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_classifying_denoising_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs'], 32, 
            best_params['lr'], best_params['noise_level'], best_params['classification_weight']
        )
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        gb = GradientBoostingClassifier(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            subsample=best_params['subsample'],
            min_samples_split=best_params['min_samples_split'],
            min_samples_leaf=best_params['min_samples_leaf'],
            random_state=42
        )
        gb.fit(X_train_latent, y_train)
        
        y_pred_proba = gb.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7197
ROC-AUC autoencoded: 0.7111
ROC-AUC autoencoded: 0.7318
ROC-AUC autoencoded: 0.7149
ROC-AUC autoencoded: 0.6832
ROC-AUC autoencoded: 0.7318
ROC-AUC autoencoded: 0.7363
ROC-AUC autoencoded: 0.7470
ROC-AUC autoencoded: 0.7426
ROC-AUC autoencoded: 0.7443
ROC-AUC autoencoded: 0.7463
ROC-AUC autoencoded: 0.7326
ROC-AUC autoencoded: 0.7406
ROC-AUC autoencoded: 0.7444
ROC-AUC autoencoded: 0.7433
ROC-AUC autoencoded: 0.6869
ROC-AUC autoencoded: 0.7050
ROC-AUC autoencoded: 0.6965
ROC-AUC autoencoded: 0.7026
ROC-AUC autoencoded: 0.7102
ROC-AUC autoencoded: 0.7749
ROC-AUC autoencoded: 0.7629
ROC-AUC autoencoded: 0.7515
ROC-AUC autoencoded: 0.7391
ROC-AUC autoencoded: 0.7507
среднее 0.73000423719005


SVC

In [119]:
def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    noise_level = trial.suggest_float('noise_level', 0.05, 0.3)
    classification_weight = trial.suggest_float('classification_weight', 0.1, 0.9)
    
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingDenoisingAutoencoder(X_train.shape[1], latent_dim, hidden_dim, dropout_rate)
        autoencoder = train_classifying_denoising_autoencoder(autoencoder, X_train, y_train, epochs, batch_size, lr, noise_level, classification_weight)
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), X_pgs_train], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), X_pgs_val], axis=1)
        
        svc = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            degree=degree,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingDenoisingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
autoencoder = train_classifying_denoising_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs'], 32, 
    best_params['lr'], best_params['noise_level'], best_params['classification_weight']
)

X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train_all), X_train_pgs], axis=1)
X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val_all), X_val_pgs], axis=1)

svc = SVC(
    C=best_params['C'],
    kernel=best_params['kernel'],
    gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
    degree=best_params['degree'] if 'degree' in best_params else 3,
    probability=True,
    random_state=42
)
svc.fit(X_train_latent, y_all_train)

y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 03:53:58,133] A new study created in memory with name: no-name-87e46670-706c-46e3-90ae-8586320739c0
[I 2025-05-11 03:54:15,549] Trial 0 finished with value: 0.5835537918871252 and parameters: {'latent_dim': 83, 'hidden_dim': 226, 'lr': 0.002634042789192513, 'epochs': 27, 'dropout_rate': 0.28213824195793613, 'noise_level': 0.09616788941691938, 'classification_weight': 0.4169829507267572, 'C': 73.92612632581356, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2}. Best is trial 0 with value: 0.5835537918871252.
[I 2025-05-11 03:54:43,934] Trial 1 finished with value: 0.5629936354574036 and parameters: {'latent_dim': 83, 'hidden_dim': 198, 'lr': 0.005073236980957397, 'epochs': 43, 'dropout_rate': 0.16151495971222773, 'noise_level': 0.23878194672915126, 'classification_weight': 0.7949620822466578, 'C': 42.744007615228846, 'kernel': 'linear'}. Best is trial 0 with value: 0.5835537918871252.
[I 2025-05-11 03:54:55,957] Trial 2 finished with value: 0.6674047235641439 and parameters

ROC-AUC autoencoded bigset: 0.7677


In [120]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingDenoisingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim'], best_params['dropout_rate'])
        autoencoder = train_classifying_denoising_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs'], 32, 
            best_params['lr'], best_params['noise_level'], best_params['classification_weight']
        )
        
        X_train_latent = np.concatenate([get_latent_features(autoencoder, X_train), train_pgs], axis=1)
        X_val_latent = np.concatenate([get_latent_features(autoencoder, X_val), test_pgs], axis=1)
        
        svc = SVC(
            C=best_params['C'],
            kernel=best_params['kernel'],
            gamma=best_params['gamma'] if 'gamma' in best_params else 'scale',
            degree=best_params['degree'] if 'degree' in best_params else 3,
            probability=True,
            random_state=42
        )
        svc.fit(X_train_latent, y_train)
        
        y_pred_proba = svc.predict_proba(X_val_latent)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7330
ROC-AUC autoencoded: 0.7114
ROC-AUC autoencoded: 0.7151
ROC-AUC autoencoded: 0.7121
ROC-AUC autoencoded: 0.7011
ROC-AUC autoencoded: 0.7247
ROC-AUC autoencoded: 0.7477
ROC-AUC autoencoded: 0.7464
ROC-AUC autoencoded: 0.7633
ROC-AUC autoencoded: 0.7616
ROC-AUC autoencoded: 0.7286
ROC-AUC autoencoded: 0.7296
ROC-AUC autoencoded: 0.7581
ROC-AUC autoencoded: 0.7659
ROC-AUC autoencoded: 0.7151
ROC-AUC autoencoded: 0.6794
ROC-AUC autoencoded: 0.6714
ROC-AUC autoencoded: 0.6717
ROC-AUC autoencoded: 0.6722
ROC-AUC autoencoded: 0.6462
ROC-AUC autoencoded: 0.7391
ROC-AUC autoencoded: 0.7441
ROC-AUC autoencoded: 0.7490
ROC-AUC autoencoded: 0.7349
ROC-AUC autoencoded: 0.7431
среднее 0.7225894100378625


MLP

In [121]:
class CombinedClassifier(nn.Module):
    def __init__(self, autoencoder, pgs_input_dim, hidden_dim=64, dropout_rate=0.2):
        super(CombinedClassifier, self).__init__()
        self.autoencoder = autoencoder
        self.pgs_branch = nn.Sequential(
            nn.Linear(pgs_input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2)
        )
        latent_dim = list(autoencoder.classifier[0].parameters())[0].shape[1]
        self.combined_classifier = nn.Sequential(
            nn.Linear(hidden_dim // 2 + latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x_snp, x_pgs):
        _, latent, _ = self.autoencoder(x_snp)
        pgs_features = self.pgs_branch(x_pgs)
        combined = torch.cat([latent, pgs_features], dim=1)
        output = self.combined_classifier(combined)
        return output

def train_combined_classifier(model, X_train_snp, X_train_pgs, y_train, epochs_freeze, epochs_unfreeze, batch_size, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    X_train_snp_tensor = torch.FloatTensor(X_train_snp).to(device)
    X_train_pgs_tensor = torch.FloatTensor(X_train_pgs).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_snp_tensor, X_train_pgs_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    criterion = nn.BCELoss()
    
    for param in model.autoencoder.parameters():
        param.requires_grad = False
    
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    model.train()
    for epoch in range(epochs_freeze):
        for batch_snp, batch_pgs, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_snp, batch_pgs)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    for param in model.autoencoder.parameters():
        param.requires_grad = True
    
    optimizer = optim.Adam(model.parameters(), lr=lr/10)
    
    for epoch in range(epochs_unfreeze):
        for batch_snp, batch_pgs, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_snp, batch_pgs)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    return model

def predict_combined(model, X_snp, X_pgs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    X_snp_tensor = torch.FloatTensor(X_snp).to(device)
    X_pgs_tensor = torch.FloatTensor(X_pgs).to(device)
    
    with torch.no_grad():
        predictions = model(X_snp_tensor, X_pgs_tensor)
    
    return predictions.cpu().numpy().flatten()

def objective(trial, X_train, y_train, X_val, y_val, X_pgs_train, X_pgs_val):
    latent_dim = trial.suggest_int('latent_dim', 10, 100)
    hidden_dim_ae = trial.suggest_int('hidden_dim_ae', 64, 256)
    hidden_dim_combined = trial.suggest_int('hidden_dim_combined', 32, 128)
    lr_ae = trial.suggest_float('lr_ae', 1e-4, 1e-2, log=True)
    lr_combined = trial.suggest_float('lr_combined', 1e-4, 1e-2, log=True)
    batch_size = 32
    epochs_ae = trial.suggest_int('epochs_ae', 10, 50)
    epochs_freeze = trial.suggest_int('epochs_freeze', 5, 20)
    epochs_unfreeze = trial.suggest_int('epochs_unfreeze', 5, 20)
    dropout_rate_ae = trial.suggest_float('dropout_rate_ae', 0.1, 0.5)
    dropout_rate_combined = trial.suggest_float('dropout_rate_combined', 0.1, 0.5)
    noise_level = trial.suggest_float('noise_level', 0.05, 0.3)
    classification_weight = trial.suggest_float('classification_weight', 0.1, 0.9)
    
    rocs = []
    for i in range(3):
        autoencoder = ClassifyingDenoisingAutoencoder(X_train.shape[1], latent_dim, hidden_dim_ae, dropout_rate_ae)
        autoencoder = train_classifying_denoising_autoencoder(autoencoder, X_train, y_train, epochs_ae, batch_size, lr_ae, noise_level, classification_weight)
        
        combined_model = CombinedClassifier(autoencoder, X_pgs_train.shape[1], hidden_dim_combined, dropout_rate_combined)
        combined_model = train_combined_classifier(combined_model, X_train, X_pgs_train, y_train, epochs_freeze, epochs_unfreeze, batch_size, lr_combined)
        
        y_pred_proba = predict_combined(combined_model, X_val, X_pgs_val)
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        rocs.append(roc_auc)
    return np.mean(rocs)

# Блок 1: оптимизация на валидации
ss = StandardScaler()
X_train_all = ss.fit_transform(X_train_all_)
X_val_all = ss.transform(X_val_all_)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all, y_all_train, X_val_all, y_val, X_train_pgs, X_val_pgs), n_trials=20)

best_params = study.best_params
autoencoder = ClassifyingDenoisingAutoencoder(X_train_all.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
autoencoder = train_classifying_denoising_autoencoder(
    autoencoder, X_train_all, y_all_train, best_params['epochs_ae'], 32, 
    best_params['lr_ae'], best_params['noise_level'], best_params['classification_weight']
)

combined_model = CombinedClassifier(autoencoder, X_train_pgs.shape[1], best_params['hidden_dim_combined'], best_params['dropout_rate_combined'])
combined_model = train_combined_classifier(
    combined_model, X_train_all, X_train_pgs, y_all_train, 
    best_params['epochs_freeze'], best_params['epochs_unfreeze'], 32, best_params['lr_combined']
)

y_pred_proba = predict_combined(combined_model, X_val_all, X_val_pgs)
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded bigset: {roc_auc:.4f}')


[I 2025-05-11 03:59:35,582] A new study created in memory with name: no-name-8bf3e2aa-e31c-4499-bf67-af93a96c1a53
[I 2025-05-11 03:59:57,321] Trial 0 finished with value: 0.6283835595429799 and parameters: {'latent_dim': 77, 'hidden_dim_ae': 205, 'hidden_dim_combined': 98, 'lr_ae': 0.00602733847227019, 'lr_combined': 0.0003566386074562373, 'epochs_ae': 29, 'epochs_freeze': 12, 'epochs_unfreeze': 9, 'dropout_rate_ae': 0.36157359442578163, 'dropout_rate_combined': 0.31587361653208434, 'noise_level': 0.17488813183509305, 'classification_weight': 0.2785149283570011}. Best is trial 0 with value: 0.6283835595429799.
[I 2025-05-11 04:00:25,545] Trial 1 finished with value: 0.6097308488612836 and parameters: {'latent_dim': 63, 'hidden_dim_ae': 97, 'hidden_dim_combined': 68, 'lr_ae': 0.002535433722614311, 'lr_combined': 0.0003699557221832689, 'epochs_ae': 39, 'epochs_freeze': 14, 'epochs_unfreeze': 16, 'dropout_rate_ae': 0.30592481959699125, 'dropout_rate_combined': 0.33787007863477686, 'noise_

ROC-AUC autoencoded bigset: 0.7637


In [122]:
# Блок 2: кросс-валидация
rocs = []
for i in range(1, 6):
    train_snp = train_snps[i - 1]
    test_snp = test_snps[i - 1]
    train_pgs = train_pgss[i - 1]
    test_pgs = test_pgss[i - 1]

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_test = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)
    
    for j in range(5):
        autoencoder = ClassifyingDenoisingAutoencoder(X_train.shape[1], best_params['latent_dim'], best_params['hidden_dim_ae'], best_params['dropout_rate_ae'])
        autoencoder = train_classifying_denoising_autoencoder(
            autoencoder, X_train, y_train, best_params['epochs_ae'], 32, 
            best_params['lr_ae'], best_params['noise_level'], best_params['classification_weight']
        )
        
        combined_model = CombinedClassifier(autoencoder, train_pgs.shape[1], best_params['hidden_dim_combined'], best_params['dropout_rate_combined'])
        combined_model = train_combined_classifier(
            combined_model, X_train, train_pgs, y_train, 
            best_params['epochs_freeze'], best_params['epochs_unfreeze'], 32, best_params['lr_combined']
        )
        
        y_pred_proba = predict_combined(combined_model, X_val, test_pgs)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        rocs.append(roc_auc)
        print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))


ROC-AUC autoencoded: 0.7603
ROC-AUC autoencoded: 0.7360
ROC-AUC autoencoded: 0.7436
ROC-AUC autoencoded: 0.7532
ROC-AUC autoencoded: 0.7443
ROC-AUC autoencoded: 0.7732
ROC-AUC autoencoded: 0.7621
ROC-AUC autoencoded: 0.7627
ROC-AUC autoencoded: 0.7788
ROC-AUC autoencoded: 0.7668
ROC-AUC autoencoded: 0.7210
ROC-AUC autoencoded: 0.7256
ROC-AUC autoencoded: 0.7249
ROC-AUC autoencoded: 0.7310
ROC-AUC autoencoded: 0.7383
ROC-AUC autoencoded: 0.7035
ROC-AUC autoencoded: 0.6903
ROC-AUC autoencoded: 0.7125
ROC-AUC autoencoded: 0.7125
ROC-AUC autoencoded: 0.6975
ROC-AUC autoencoded: 0.7779
ROC-AUC autoencoded: 0.7590
ROC-AUC autoencoded: 0.7662
ROC-AUC autoencoded: 0.7574
ROC-AUC autoencoded: 0.7547
среднее 0.7421353944635248
